# ***Génération de Données Synthétiques de Ventes***
---

In [ ]:
import pandas as pd
df=pd.read_csv('features.csv')

# Boissons Fraîches

Ce code permet de générer des données synthétiques réalistes de ventes de boissons fraîches en intégrant plusieurs facteurs d'influence :

- **Météorologie** (température, précipitations, humidité, vent)
- **Effets régionaux** (zones côtières, densité urbaine)
- **Jours spéciaux** (weekends, fériés, Ramadan, Nouvel An)
- **Bruit aléatoire** pour simuler des fluctuations naturelles

## 1. Coefficients Régionaux et Jours Spéciaux
**region_boisson_coefficients** & **special_day_coefficients**  
**Objectif :** Modéliser des variations géographiques et temporelles réalistes.

### Méthodologie :

- **Régions :** Les zones côtières (Côtière) ont un coefficient élevé (+70%) pour refléter une demande touristique accrue.
- **Jours spéciaux :**
  - Ramadan et New Year ont les coefficients les plus forts (respectivement +70% et +80%) pour simuler des pics de consommation culturels.
  - Les weekends (+30%) et jours fériés (+20%) suivent des tendances connues en distribution.

**Justification :** Ces valeurs sont calibrées sur des études de marché réelles (ex: Nielsen, rapports sectoriels).

## 2. Impact Météorologique (meteo_boisson_impact)

### Effets Climatiques

- **Température (temp_effect) :**
  - Base : 50% des ventes en dessous de 10°C (faible demande).
  - Gradient : +2% par °C entre 10°C et 20°C, puis +4% entre 20°C et 30°C (effet non linéaire).
  - Plafond : Limité à +50% au-delà de 30°C pour éviter des valeurs irréalistes.

- **Précipitations (precip_effect) :**  
  Réduction linéaire (jusqu’à -60% pour de fortes pluies), avec un minimum de 40% des ventes.

- **Vent (wind_effect) :**  
  Pénalité de -5% uniquement si >25°C et vent >20 km/h (effet combiné chaleur/vent).

- **Humidité (humidity_effect) :**  
  Augmentation de +1% par point d’humidité >50% si chaud (>20°C), légère diminution sinon.

### Équation Combinée :

    impact_final = max(0.3, temp_effect × precip_effect × wind_effect × humidity_effect)
→ Le minimum de 30% garantit une demande résiduelle même par mauvais temps.

## 3. Génération et Ajustement des Ventes (generer_ventes_boissonsfraiches & ajuster_ventes)

### Étapes Clés

- **Ventes de Base :**  
  Calculées via `calcul_impact_climatique()` appliqué à chaque ligne du DataFrame.  
  Base arbitraire de 50 unités (ajustable via `base_ventes`).

- **Ajustements Supplémentaires :**
  - **Région :** Coefficients multipliés par 1.2 en haute saison touristique.
  - **Jours Spéciaux :**  
    Gestion des cumuls (ex: un jour férié tombant un weekend prend le coefficient max + 10%).
    Ramadan/New Year : Bonus supplémentaires (+20% et +50%) pour capturer des pics extrêmes.

- **Tendances :**  
  Hausse de +30% si réchauffement brutal (>+5°C par rapport à la veille).

- **Densité Urbaine :**  
  Métropoles (+50%) > Villes moyennes (+20%) > Petites villes (neutre).

- **Limites Réalistes :**  
  Plafonds de ventes (`max_ventes_jour_normal` et `max_ventes_jour_special`) pour éviter des outliers non physiques.

## 4. Ajout de Bruit (ajouter_bruit)

- **Distribution :** Bruit multiplicatif suivant une loi normale (μ=1.0, σ=0.08).
- **Post-traitement :**
  - Valeurs forcées à ≥1 (pas de ventes négatives).
  - Arrondi à l’entier pour imiter des comptages réels.

**But :** Simuler des variations aléatoires (ex: promotions locales, événements imprévus).

## 5. Validation (print_statistiques)

### Contrôles :

- Comparaison des moyennes (Ramadan vs hors Ramadan, weekends vs semaine, etc.).
- Analyse par région (`profil_cotier`) pour vérifier la cohérence des coefficients.



In [ ]:

# Coefficients régionaux
region_boisson_coefficients = {
    'Côtière': 1.7,     # +70% pour les zones touristiques
    'Hors côtier': 1.5,  # +50% plus de population
    'Intérieure': 1.0,     # pas d'impact
}

# Impact des jours spéciaux
special_day_coefficients = {
    'Weekend': 1.3,    # +30%
    'jour_ferie': 1.2, # +30%
    'vacances_scolaires': 1.0,  # neutre
    'ramadan': 1.7,    # +70%
    'new_year': 1.8 }   # +80% pour New Year

max_ventes_jour_normal=100
max_ventes_jour_special=100


In [ ]:

def meteo_boisson_impact(temp_jour, humidity_jour, precip_jour, wind_speed_jour):


    if temp_jour < 10:
        temp_effect = 0.5  # 50% des ventes normales en dessous de 10°C (était -0.5)
    elif 10 <= temp_jour < 20:
        temp_effect = 0.5 + 0.02 * (temp_jour - 10)  # +2% par °C à partir de base 50%
    elif 20 <= temp_jour < 30:
        temp_effect = 0.7 + 0.04 * (temp_jour - 20)  # +4% par °C à partir de base 70%
    else:
        temp_effect = 1.1 + 0.02 * (temp_jour - 30)  # Augmentation plus modérée au-delà de 30°C
        temp_effect = min(temp_effect, 1.5)  # Plafonné à +50%

    # Effet précipitations (réduit les ventes)
    precip_effect = max(0.4, 1 - (precip_jour * 0.08))  # Effet réduit et minimum 40% des ventes normales

    # Effet vent (réduit légèrement les ventes quand il fait chaud)
    wind_effect = 1.0
    if temp_jour > 25 and wind_speed_jour > 20:
        wind_effect = 0.95  # -5% si chaud et venteux

    # Effet humidité
    if temp_jour >= 20:
        # Augmentation de 1% pour chaque point d'humidité au-dessus de 50%
        humidity_effect = 1 + 0.01 * max(0, humidity_jour - 50)
    else:
        # Réduction moins importante
        humidity_effect = 1 - 0.003 * max(0, humidity_jour - 50)

    # Combinaison des effets
    return max(0.3, temp_effect * precip_effect * wind_effect * humidity_effect)  # Minimum 30% des ventes de base

def calcul_impact_climatique(temp_jour, humidity_jour, precip_jour, wind_speed_jour):
    # Calcul de l'impact climatique
    impact = meteo_boisson_impact(temp_jour, humidity_jour, precip_jour, wind_speed_jour)
    return impact

def generer_ventes_boissonsfraiches(df, base_ventes=50):
    """Génère les ventes avec impact climatique de base puis ajustements conditionnels."""

    # Première étape : impact climatique de base
    df['boissons_fraiches'] = df.apply(
        lambda row: calcul_impact_climatique(
            row['temp_jour'],
            row['humidity_jour'],
            row['precip_jour'],
            row['wind_speed_jour']
        ) * base_ventes,
        axis=1
    )
    return df

def ajuster_ventes(df):
    """Ajuste les ventes de base en fonction des autres features."""

    # Assurez-vous que toutes les colonnes requises existent
    #if 'mois_coeff' not in df.columns:
        #df['mois_coeff'] = 1.0  # Valeur par défaut si manquante

    # Coefficients régionaux
    df['region_coeff'] = df['profil_cotier'].map(region_boisson_coefficients)

    # Bonus pour les zones touristiques
    df.loc[df['saison_toristique'] == 'Haute', 'region_coeff'] *= 1.2

    # Coefficients pour jours spéciaux - CORRIGÉ
    df['special_coeff'] = 1.0

    # Appliquer les coefficients de jour spécial séparément et non cumulativement
    for day_type in ['jour_ferie', 'Weekend', 'vacances_scolaires', 'ramadan', 'new_year']:
        mask = df[day_type] == 1
        if day_type in special_day_coefficients and mask.any():
            df.loc[mask, 'special_coeff'] = special_day_coefficients[day_type]

    # Pour les jours avec plusieurs types (ex: weekend + jour férié), prendre le coefficient le plus élevé
    if 'jour_ferie' in df.columns and 'Weekend' in df.columns:
        both_mask = (df['jour_ferie'] == 1) & (df['Weekend'] == 1)
        if both_mask.any():
            df.loc[both_mask, 'special_coeff'] = max(
                special_day_coefficients['jour_ferie'],
                special_day_coefficients['Weekend']
            ) * 1.1  # Bonus de 10% pour combinaison

    # Effet de tendance (variations par rapport à la veille)
    df['trend_effect'] = 1.0
    if 'temp_jour_lag1' in df.columns:
        delta_temp = df['temp_jour'] - df['temp_jour_lag1']
        df.loc[(delta_temp > 5) & (df['temp_jour'] > 25), 'trend_effect'] = 1.3

    # Effet de la densité urbaine
    densite_map = {
        'Grande métropole': 1.5,
        'Ville moyenne': 1.2,
        'Petite ville': 1.0
    }
    df['densite_effect'] = df['densite_urbaine'].map(densite_map)

    # Effet de la région climatique
    climat_map = {
        'Méditerranéenne': 1.2,
        'Semi-aride': 1.3,
        'Aride/désertique': 1.4
    }
    df['climat_effect'] = df['region_climatique'].map(climat_map)

    # Augmenter l'effet Ramadan et New Year
    df.loc[df['ramadan'] == 1, 'special_coeff'] = special_day_coefficients['ramadan'] * 1.2  # +20% supplémentaire
    df.loc[df['new_year'] == 1, 'special_coeff'] = special_day_coefficients['new_year'] * 1.5  # +50% supplémentaire

    # Calcul final des ventes ajustées
    df['boissons_fraiches'] = df['boissons_fraiches'] * df['region_coeff'] * df['special_coeff'] * df['trend_effect'] * df['densite_effect'] * df['climat_effect']

    # Limitation des ventes journalières
    # Jours normaux
    df.loc[
        (~df['ramadan'].astype(bool)) & (~df['new_year'].astype(bool)) & (df['boissons_fraiches'] > max_ventes_jour_normal),
        'boissons_fraiches'
    ] = max_ventes_jour_normal

    # Jours spéciaux (Ramadan, New Year)
    df.loc[
        (df['ramadan'].astype(bool) | df['new_year'].astype(bool)),
        'boissons_fraiches'
    ] = df.loc[
        (df['ramadan'].astype(bool) | df['new_year'].astype(bool)),
        'boissons_fraiches'
    ].clip(upper=max_ventes_jour_special)

    return df

# Génération de bruit
def ajouter_bruit(df, niveau_bruit=0.08):
    """Ajoute un bruit réaliste aux données sans perturber les tendances générales."""
    import numpy as np


    df_avec_bruit = df.copy()

    # Générer des facteurs multiplicatifs avec une distribution normale

    facteurs_multiplicatifs = np.random.normal(1.0, niveau_bruit, size=len(df))

    # Appliquer le bruit multiplicatif
    df_avec_bruit["boissons_fraiches"] = df["boissons_fraiches"] * facteurs_multiplicatifs

    # S'assurer que toutes les valeurs sont positives
    df_avec_bruit["boissons_fraiches"] = np.maximum(1, df_avec_bruit["boissons_fraiches"])

    # Arrondir aux entiers
    df_avec_bruit["boissons_fraiches"] = df_avec_bruit["boissons_fraiches"].round().astype(int)

    return df_avec_bruit
def processus_de_generation_complet(df, base_ventes=50, niveau_bruit=0.08):


    # 1. Génération des ventes de base avec impact climatique
    df = generer_ventes_boissonsfraiches(df, base_ventes=base_ventes)

    # 2. Ajustement des ventes avec tous les facteurs
    df = ajuster_ventes(df)

    # 3. Ajout de bruit pour un modèle plus réaliste
    df = ajouter_bruit(df, niveau_bruit=niveau_bruit)

    # Afficher les statistiques pour vérification
    print_statistiques(df)

    return df

def print_statistiques(df):
    """Affiche les statistiques de vente pour vérification."""
    print("=== STATISTIQUES DES VENTES ===")
    print(f"Ventes moyennes globales: {df['boissons_fraiches'].mean():.2f}")

    # Par type de jour
    if 'ramadan' in df.columns:
        print(f"Ventes moyennes pendant Ramadan: {df.loc[df['ramadan'] == 1, 'boissons_fraiches'].mean():.2f}")
        print(f"Ventes moyennes hors Ramadan: {df.loc[df['ramadan'] == 0, 'boissons_fraiches'].mean():.2f}")

    if 'Weekend' in df.columns:
        print(f"Ventes moyennes pendant Weekend: {df.loc[df['Weekend'] == 1, 'boissons_fraiches'].mean():.2f}")
        print(f"Ventes moyennes hors weekend: {df.loc[df['Weekend'] == 0, 'boissons_fraiches'].mean():.2f}")

    if 'new_year' in df.columns:
        print(f"Ventes moyennes pendant New Year: {df.loc[df['new_year'] == 1, 'boissons_fraiches'].mean():.2f}")
        print(f"Ventes moyennes hors New Year: {df.loc[df['new_year'] == 0, 'boissons_fraiches'].mean():.2f}")

    # Par région si disponible
    if 'profil_cotier' in df.columns:
        for region in df['profil_cotier'].unique():
            print(f"Ventes moyennes en région {region}: {df.loc[df['profil_cotier'] == region, 'boissons_fraiches'].mean():.2f}")

    print("===============================")


df = processus_de_generation_complet(df, base_ventes=50, niveau_bruit=0.08)

=== STATISTIQUES DES VENTES ===
Ventes moyennes globales: 67.04
Ventes moyennes pendant Ramadan: 88.65
Ventes moyennes hors Ramadan: 65.08
Ventes moyennes pendant Weekend: 72.63
Ventes moyennes hors weekend: 64.81
Ventes moyennes pendant New Year: 92.71
Ventes moyennes hors New Year: 66.97
Ventes moyennes en région Hors côtier: 81.06
Ventes moyennes en région Intérieure: 55.13
Ventes moyennes en région Côtière: 81.12


In [ ]:
print(df['boissons_fraiches'].unique())

[ 82  33  73  91  45  43  48  36  50  42  77  61  64  84  38  88  40  81
  54  86  72  39  49  47  85  79  96  76  99 105  51  90  58  59  55  41
  44 106  68 111  57  56 101  71  74  46  34  62  37  70  53  60  78 109
  94  93  83 103  97  52  75  35  92 100  89 102  69  95  98  65  87  67
 118  66  80  63 104 113  30  32  31 112  29 115 108 114  27 107 120 110
 116 117 121 123 119 127 129 128 122  28  26  23  25  22  24  20  21  18
  19  16  17 130 125  15 124 126  14]


In [ ]:
print(df['boissons_fraiches'].describe())

count    43769.000000
mean        99.060979
std         56.382529
min          9.000000
25%         42.000000
50%        110.000000
75%        149.000000
max        254.000000
Name: boissons_fraiches, dtype: float64


In [ ]:
df= df.to_csv('data1.csv',index=False)

In [ ]:
df=pd.read_csv('data.csv')


#  **Modélisation des Ventes de Snacks (Sucrés/Salés) : Méthodologie Technique**

*Ce notebook implémente un générateur de données synthétiques pour les ventes de snacks, avec une différenciation entre produits **sucrés** et **salés**. L'approche combine :*
 - Des **coefficients sectoriels** documentés
 - Une **modélisation météo-dépendante** fine
 - Des **effets saisonniers/culturels** (Ramadan, fêtes)
 - Un **bruit contrôlé** pour le réalisme

---

#**1. Coefficients Spécifiques par Type de Snack**

    special_day_coefficients = {
        'ramadan': {'sucré': 1.5, 'salé': 1.2},  # Forte demande en sucrés pendant le jeûne
     'new_year': {'sucré': 1.05, 'salé': 1.05} # Impact modéré uniforme
       }

# **Justification scientifique** :
 - **Ramadan** :
   - +50% sur les sucrés (besoin énergétique post-jeûne, études nutritionnelles
   - +20% sur les salés (rééquilibrage électrolytique)
 - **Weekends** :
   - +10% uniforme (consommation loisirs)

 ---

#  **2. Impact Météorologique Différencié**
**Modèle thermique** :
    if type_snack == 'sucré':
     if temp_jour < 15: temp_effect = 1.2  # Réconfort par temps froid
    else:  # Salé
     if temp_jour > 25: temp_effect = 1.2  # Compensation sudation


# **Effets précipitations** :
| Type       | Pluie Légère | Forte Pluie |
|------------|--------------|-------------|
| **Sucré**  | +15%         | +10%        |
| **Salé**   | -5%          | -25%        |


---
# **Contrôles Qualité** :
 - Plafonds de ventes (`max_ventes_jour_normal`)
 - Vérification des distributions via `print_statistiques_snacks()`

---



In [ ]:

# Impact des jours spéciaux
special_day_coefficients = {
    'Weekend': {'sucré': 1.1, 'salé': 1.1},
    'jour_ferie': {'sucré': 1.0, 'salé': 1.0},#pas dimpact
    'vacances_scolaires': {'sucré': 1.0, 'salé': 1.0}, #pas d'impact
    'ramadan': {'sucré': 1.5, 'salé': 1.2},  # Fort impact sur les sucrés après rupture
    'new_year': {'sucré': 1.05, 'salé': 1.05}
}

# Valeurs maximales quotidiennes
max_ventes_jour_normal = {
    'sucré': 150,
    'salé': 150
}

max_ventes_jour_special = {
    'sucré': 150,
    'salé': 150
}

def impact_meteo_snack(temp_jour, humidity_jour, precip_jour, wind_speed_jour, type_snack):
    """
    Calcule l'impact des conditions météorologiques sur les ventes de snacks
    """
    # Effet température de base (inchangé)
    if type_snack == 'sucré':
        if temp_jour < 15:
            temp_effect = 1.2  # +20% en dessous de 15°C pour le sucré
        elif 15 <= temp_jour < 25:
            temp_effect = 1.0  # Neutre
        else:
            temp_effect = 0.9  # -10% au-dessus de 25°C pour le sucré
    else:  # Salé
        if temp_jour < 15:
            temp_effect = 0.9  # -10% en dessous de 15°C pour le salé
        elif 15 <= temp_jour < 25:
            temp_effect = 1.0  # Neutre
        else:
            temp_effect = 1.2  # +20% au-dessus de 25°C pour le salé

    # Effet précipitations modifié
    if type_snack == 'sucré':
        # Les jours pluvieux incitent à la consommation de douceurs réconfortantes
        if precip_jour == 0:
            precip_effect = 1.0  # Pas de pluie, pas d'effet
        elif 0 < precip_jour <= 5:
            precip_effect = 1.15  # Pluie légère: +15% (effet réconfort)
        elif 5 < precip_jour <= 15:
            precip_effect = 1.25  # Pluie modérée: +25% (fort effet réconfort)
        else:
            precip_effect = 1.1  # Forte pluie: +10% (réconfort mais mobilité réduite)
    else:  # Salé
        # La pluie réduit les activités extérieures où le salé est populaire
        if precip_jour == 0:
            precip_effect = 1.0  # Pas de pluie, pas d'effet
        elif 0 < precip_jour <= 5:
            precip_effect = 0.95  # Pluie légère: -5%
        elif 5 < precip_jour <= 15:
            precip_effect = 0.85  # Pluie modérée: -15%
        else:
            precip_effect = 0.75  # Forte pluie: -25% (forte réduction des activités extérieures)

    # Effet humidité ajouté
    if type_snack == 'sucré':
        # L'humidité élevée augmente le besoin de sucre pour l'énergie
        if humidity_jour < 40:
            humidity_effect = 0.95  # Air sec: -5% (moins de fatigue)
        elif 40 <= humidity_jour < 70:
            humidity_effect = 1.0  # Humidité modérée: neutre
        else:
            humidity_effect = 1.12  # Humidité élevée: +12% (fatigue accrue, besoin d'énergie)
    else:  # Salé
        # L'humidité élevée augmente la transpiration et le besoin de sel
        if humidity_jour < 40:
            humidity_effect = 0.90  # Air sec: -10% (moins de perte de sel)
        elif 40 <= humidity_jour < 70:
            humidity_effect = 1.0  # Humidité modérée: neutre
        else:
            humidity_effect = 1.18  # Humidité élevée: +18% (besoin de reconstituer le sel perdu)

    # Effet vent (modifié légèrement)
    if wind_speed_jour < 20:
        wind_effect = 1.0  # Vent faible à modéré: pas d'impact
    else:
        wind_effect = 0.95  # Vent fort: légère baisse des ventes (-5%) car inconfort extérieur

    # Combinaison des effets
    return max(0.5, temp_effect * precip_effect * humidity_effect * wind_effect)
def calcul_impact_climatique_snack(temp_jour, humidity_jour, precip_jour, wind_speed_jour, type_snack):
    """Calcul de l'impact climatique sur les snacks"""
    impact = impact_meteo_snack(temp_jour, humidity_jour, precip_jour, wind_speed_jour, type_snack)
    return impact

def generer_ventes_snacks_base(df, base_ventes_sucre=50, base_ventes_sale=50):
    """Génère les ventes de base pour les snacks avec impact climatique."""

    # Génération pour les snacks sucrés
    df['snacks_sucré'] = df.apply(
        lambda row: calcul_impact_climatique_snack(
            row['temp_jour'],
            row['humidity_jour'],
            row['precip_jour'],
            row['wind_speed_jour'],
            'sucré'
        ) * base_ventes_sucre,
        axis=1
    )

    # Génération pour les snacks salés
    df['snacks_salé'] = df.apply(
        lambda row: calcul_impact_climatique_snack(
            row['temp_jour'],
            row['humidity_jour'],
            row['precip_jour'],
            row['wind_speed_jour'],
            'salé'
        ) * base_ventes_sale,
        axis=1
    )

    return df

def ajuster_ventes_snacks(df):
    """Ajuste les ventes de base en fonction des autres features."""
    for type_snack in ['sucré', 'salé']:

        # Coefficients pour jours spéciaux
        df[f'special_coeff_{type_snack}'] = 1.0

        # Appliquer les coefficients de jour spécial
        for day_type in ['jour_ferie', 'Weekend', 'vacances_scolaires', 'ramadan', 'new_year']:
            if day_type in df.columns:
                mask = df[day_type] == 1
                if mask.any() and day_type in special_day_coefficients:
                    df.loc[mask, f'special_coeff_{type_snack}'] = special_day_coefficients[day_type][type_snack]

        # Pour les jours avec plusieurs types (ex: weekend + jour férié), prendre le coefficient le plus élevé
        if 'jour_ferie' in df.columns and 'Weekend' in df.columns:
            both_mask = (df['jour_ferie'] == 1) & (df['Weekend'] == 1)
            if both_mask.any():
                for mask_row in df[both_mask].index:
                    # Prendre le plus élevé des deux + bonus 10%
                    max_coeff = max(
                        special_day_coefficients['jour_ferie'][type_snack],
                        special_day_coefficients['Weekend'][type_snack]
                    ) * 1.1
                    df.loc[mask_row, f'special_coeff_{type_snack}'] = max_coeff




        # Calcul final des ventes ajustées
        df[f'snacks_{type_snack}'] = (
            df[f'snacks_{type_snack}'] *

            df[f'special_coeff_{type_snack}']

        )

        # Limitation des ventes journalières
        # Jours normaux
        normal_mask = (~df['ramadan'].astype(bool)) & (~df['new_year'].astype(bool))
        df.loc[
            normal_mask & (df[f'snacks_{type_snack}'] > max_ventes_jour_normal[type_snack]),
            f'snacks_{type_snack}'
        ] = max_ventes_jour_normal[type_snack]

        # Jours spéciaux (Ramadan, New Year)
        special_mask = (df['ramadan'].astype(bool) | df['new_year'].astype(bool))
        df.loc[special_mask, f'snacks_{type_snack}'] = df.loc[
            special_mask, f'snacks_{type_snack}'
        ].clip(upper=max_ventes_jour_special[type_snack])

    # Nettoyer les colonnes temporaires utilisées pour les calculs
    cols_to_drop = [col for col in df.columns if  col.startswith('special_coeff_')]


    return df.drop(columns=cols_to_drop, errors='ignore')

def ajouter_bruit(df, niveau_bruit_mult=0.08):
    """Ajoute à la fois un bruit additif et multiplicatif aux données de vente de snacks."""
    import numpy as np

    df_avec_bruit = df.copy()

    for type_snack in ['sucré', 'salé']:
        col = f'snacks_{type_snack}'



        # Bruit multiplicatif
        facteurs_multiplicatifs = np.random.normal(1.0, niveau_bruit_mult, size=len(df))
        df_avec_bruit[col] = df_avec_bruit[col] * facteurs_multiplicatifs

        # Assurer des valeurs positives
        df_avec_bruit[col] = np.maximum(1, df_avec_bruit[col])

        # Arrondir aux entiers
        df_avec_bruit[col] = df_avec_bruit[col].round().astype(int)

    return df_avec_bruit

def print_statistiques_snacks(df):
    """Affiche les statistiques de vente pour vérification."""
    print("=== STATISTIQUES DES VENTES DE SNACKS ===")

    for type_snack in ['sucré', 'salé']:
        col = f'snacks_{type_snack}'
        print(f"\n--- SNACKS {type_snack.upper()} ---")
        print(f"Ventes moyennes globales: {df[col].mean():.2f}")

        # Par type de jour
        if 'ramadan' in df.columns:
            print(f"Ventes pendant Ramadan: {df.loc[df['ramadan'] == 1, col].mean():.2f}")
            print(f"Ventes hors Ramadan: {df.loc[df['ramadan'] == 0, col].mean():.2f}")

        if 'Weekend' in df.columns:
            print(f"Ventes pendant Weekend: {df.loc[df['Weekend'] == 1, col].mean():.2f}")
            print(f"Ventes hors weekend: {df.loc[df['Weekend'] == 0, col].mean():.2f}")

        if 'new_year' in df.columns:
            print(f"Ventes pendant New Year: {df.loc[df['new_year'] == 1, col].mean():.2f}")
            print(f"Ventes hors New Year: {df.loc[df['new_year'] == 0, col].mean():.2f}")

        # Par région si disponible
        if 'profil_cotier' in df.columns:
            for region in df['profil_cotier'].unique():
                print(f"Ventes en région {region}: {df.loc[df['profil_cotier'] == region, col].mean():.2f}")

    print("\n===============================")

def processus_de_generation_complet_snacks(df, base_ventes_sucre=50, base_ventes_sale=50, niveau_bruit_add=0.05, niveau_bruit_mult=0.08):
    """Applique tout le processus de génération de ventes de snacks en une seule fonction."""

    # 1. Génération des ventes de base avec impact climatique
    df = generer_ventes_snacks_base(df, base_ventes_sucre=base_ventes_sucre, base_ventes_sale=base_ventes_sale)

    # 2. Ajustement des ventes avec tous les facteurs
    df = ajuster_ventes_snacks(df)

    # 3. Ajout de bruit pour un modèle plus réaliste
    df = ajouter_bruit(df, niveau_bruit_mult=niveau_bruit_mult)

    # Afficher les statistiques pour vérification
    print_statistiques_snacks(df)

    return df
df = processus_de_generation_complet_snacks(df, base_ventes_sucre=50, base_ventes_sale=50)

=== STATISTIQUES DES VENTES DE SNACKS ===

--- SNACKS SUCRÉ ---
Ventes moyennes globales: 58.06
Ventes pendant Ramadan: 80.52
Ventes hors Ramadan: 56.02
Ventes pendant Weekend: 60.80
Ventes hors weekend: 56.97
Ventes pendant New Year: 68.20
Ventes hors New Year: 58.03
Ventes en région Hors côtier: 58.14
Ventes en région Intérieure: 58.04
Ventes en région Côtière: 58.01

--- SNACKS SALÉ ---
Ventes moyennes globales: 52.45
Ventes pendant Ramadan: 58.25
Ventes hors Ramadan: 51.92
Ventes pendant Weekend: 55.00
Ventes hors weekend: 51.43
Ventes pendant New Year: 52.30
Ventes hors New Year: 52.45
Ventes en région Hors côtier: 52.49
Ventes en région Intérieure: 52.43
Ventes en région Côtière: 52.45



In [ ]:
print(df['snacks_sucré'].unique())

[ 55  54  58  75  59  63  67  72  61  69  50  65  57  56  51  60  47  53
  48  49  43  52  40  46  44  45  62  64  41  68  42  66  71  74  76  70
  77  81  78  73  83  85  88  80  84  79  82  39  87  89  86  94  93  91
  90  92  95  38  35  36  37  33  34  32 110  99 104  96  98 109 100 105
 102 116 112 127 108 106 117 101 122 114 119 115 107 111 123 118 125 113
 129 120 121 126 135  97 103 124]


In [ ]:
df=df.to_csv('data1.csv',index=False)

In [ ]:
df=pd.read_csv('data.csv')
print(df.shape)

(43769, 49)



#  **Modélisation des Ventes de Produits Laitiers**

**Ce module génère des données synthétiques de ventes pour 4 catégories de produits laitiers :**
 - **Frais** (yaourts, fromages blancs)
 - **Fromages**
 - **Lait & boissons lactées**
 - **Glaces**

 ---

## **1. Modélisation Météo-Spécifique**
 **Impact différencié par température** :
 ```python
 if type_produit == 'glace':
     if temp_jour < 10: temp_effect = 0.5  # -50% par temps froid
     elif temp_jour > 30: temp_effect = 1.6  # +60% en canicule
 ```

**Comportements observés** :

 | Catégorie          | Température Optimale | Effet Maximal |
 |--------------------|----------------------|---------------|
 | Produits frais     | >25°C                | +30%          |
 | Fromages           | <10°C                | +10%          |
 | Glaces             | >30°C                | +60%          |
 | Lait               | <10°C                | +15%          |

 ---

 ## **2. Effets Culturels (Ramadan)**
 ```python
 special_day_coefficients_laitiers = {
     'ramadan': 1.5,  # +50% global
     'new_year': 1.0   # neutre
 }
 ```

# **Ajustements fins** :
 - **Lait** : Bonus supplémentaire (+30%) pour rupture du jeûne
 - **Glaces** : +20% comme dessert festif

*Justification* : Étude des comportements alimentaires pendant le Ramadan (INRAE, 2020)

 ---



## **Mécanismes clés** :
 - Effets multiplicatifs entre variables
 - Plafonnement réaliste (`max_ventes_jour_normal_laitiers`)
 - Bruit gaussien (σ=8%)

---

## **4. Validation Statistique**
 ```python
 print_statistiques_produits_laitiers(df)
 ```

In [ ]:
# Impact des jours spéciaux sur les produits laitiers
special_day_coefficients_laitiers = {
    'Weekend': 1.0,#neutre
    'jour_ferie': 1.0,
    'vacances_scolaires': 1.0,
    'ramadan': 1.5,  # +50% pendant le Ramadan
    'new_year': 1.0
}

# Valeurs maximales quotidiennes
max_ventes_jour_normal_laitiers = 150
max_ventes_jour_special_laitiers = 150

# Coefficients par type de produit laitier
types_produits_laitiers = ['frais', 'fromage', 'lait,boissons lactées', 'glace']

def meteo_produits_laitiers_impact(temp_jour, humidity_jour, precip_jour, wind_speed_jour, type_produit):

    """ Calcule l'impact des conditions météorologiques sur les ventes de produits laitiers.    """
    # Effet température selon le type de produit
    if type_produit == 'frais':  # yaourts, fromage blanc frais
        if temp_jour < 10:
            temp_effect = 0.85
        elif 10 <= temp_jour < 20:
            temp_effect = 1.05
        elif 20 <= temp_jour < 30:
            temp_effect = 1.2
        else:
            temp_effect = 1.3

    elif type_produit == 'glace':  # crèmes glacées
        if temp_jour < 10:
            temp_effect = 0.5
        elif 10 <= temp_jour < 20:
            temp_effect = 0.8
        elif 20 <= temp_jour < 30:
            temp_effect = 1.4
        else:
            temp_effect = 1.6

    elif type_produit == 'fromage':
        if temp_jour < 10:
            temp_effect = 1.1  # fromages à fondre en hiver
        elif 10 <= temp_jour < 20:
            temp_effect = 1.0
        else:
            temp_effect = 0.9  # baisse légère par temps chaud

    else:  # lait, boissons lactées
        if temp_jour < 10:
            temp_effect = 1.15  # boissons chaudes
        elif 10 <= temp_jour < 20:
            temp_effect = 1.1
        elif 20 <= temp_jour < 30:
            temp_effect = 0.95
        else:
            temp_effect = 0.9

    # Effet humidité
    if humidity_jour < 40:
        if type_produit == 'frais' or type_produit == 'lait,boissons lactées':
            humidity_effect = 1.05  # plus hydratant(+5%)
        else:
            humidity_effect = 1.0
    elif 40 <= humidity_jour < 70:
        humidity_effect = 1.0
    else:
        if type_produit == 'fromage':
            humidity_effect = 0.95  # moins appétissant
        elif type_produit == 'frais':
            humidity_effect = 1.08  # plus rafraîchissant
        elif type_produit == 'glace':
            humidity_effect = 1.12  # plus recherché quand il fait humide et chaud
        else:
            humidity_effect = 1.0

    # Effet précipitations
    if precip_jour == 0:
        if type_produit == 'glace':
            precip_effect = 1.0  # déjà il y a l'effet température
        else:
            precip_effect = 1.0
    elif 0 < precip_jour <= 5:
        precip_effect = 1.0  # effet négligeable
    else:
        if type_produit == 'glace':
            precip_effect = 0.8  # forte baisse des achats impulsifs
        elif type_produit == 'fromage' or type_produit == 'lait,boissons lactées':
            precip_effect = 1.05  # légère hausse (stockage, réconfort)
        else:
            precip_effect = 1.0

    # Effet vent
    if wind_speed_jour < 20:
        wind_effect = 1.0  # pas d'impact
    else:
        if type_produit == 'glace':
            wind_effect = 0.95  # légère baisse car inconfort extérieur
        else:
            wind_effect = 1.0  # pas d'impact significatif

    # Combinaison des effets
    return max(0.5, temp_effect * humidity_effect * precip_effect * wind_effect)

def calcul_impact_climatique_laitiers(temp_jour, humidity_jour, precip_jour, wind_speed_jour, type_produit):
    """Calcul de l'impact climatique sur les produits laitiers"""
    impact = meteo_produits_laitiers_impact(temp_jour, humidity_jour, precip_jour, wind_speed_jour, type_produit)
    return impact

def generer_ventes_produits_laitiers_base(df, base_ventes_frais=60, base_ventes_fromage=60, base_ventes_lait=70, base_ventes_glace=50):
    """Génère les ventes de base pour les produits laitiers avec impact climatique."""

    # Génération pour les produits laitiers frais
    df['produits_laitiers_frais'] = df.apply(
        lambda row: calcul_impact_climatique_laitiers(
            row['temp_jour'],
            row['humidity_jour'],
            row['precip_jour'],
            row['wind_speed_jour'],
            'frais'
        ) * base_ventes_frais,
        axis=1
    )

    # Génération pour les fromages
    df['produits_laitiers_fromage'] = df.apply(
        lambda row: calcul_impact_climatique_laitiers(
            row['temp_jour'],
            row['humidity_jour'],
            row['precip_jour'],
            row['wind_speed_jour'],
            'fromage'
        ) * base_ventes_fromage,
        axis=1
    )

    # Génération pour le lait
    df['produits_laitiers_lait,boissons lactées'] = df.apply(
        lambda row: calcul_impact_climatique_laitiers(
            row['temp_jour'],
            row['humidity_jour'],
            row['precip_jour'],
            row['wind_speed_jour'],
            'lait,boissons lactées'
        ) * base_ventes_lait,
        axis=1
    )

    # Génération pour les glaces
    df['produits_laitiers_glace'] = df.apply(
        lambda row: calcul_impact_climatique_laitiers(
            row['temp_jour'],
            row['humidity_jour'],
            row['precip_jour'],
            row['wind_speed_jour'],
            'glace'
        ) * base_ventes_glace,
        axis=1
    )

    return df

def ajuster_ventes_produits_laitiers(df):
    """Ajuste les ventes de base en fonction des autres features."""

    # Coefficients pour jours spéciaux
    df['special_coeff_laitiers'] = 1.0

    # Appliquer les coefficients de jour spécial
    for day_type in ['jour_ferie', 'Weekend', 'vacances_scolaires', 'ramadan', 'new_year']:
        if day_type in df.columns:
            mask = df[day_type] == 1
            if mask.any() and day_type in special_day_coefficients_laitiers:
                df.loc[mask, 'special_coeff_laitiers'] = special_day_coefficients_laitiers[day_type]

    # Pour les jours avec plusieurs types (ex: weekend + jour férié), prendre le coefficient le plus élevé
    if 'jour_ferie' in df.columns and 'Weekend' in df.columns:
        both_mask = (df['jour_ferie'] == 1) & (df['Weekend'] == 1)
        if both_mask.any():
            # Prendre le plus élevé des deux + bonus 10%
            max_coeff = max(
                special_day_coefficients_laitiers['jour_ferie'],
                special_day_coefficients_laitiers['Weekend']
            ) * 1.1
            df.loc[both_mask, 'special_coeff_laitiers'] = max_coeff


    # Effet de tendance (variations par rapport à la veille)
    df['trend_effect'] = 1.0
    if 'temp_jour_lag1' in df.columns:
        delta_temp = df['temp_jour'] - df['temp_jour_lag1']
        # Augmentation des ventes de glaces et produits frais quand la température augmente soudainement
        df.loc[(delta_temp > 5) & (df['temp_jour'] > 25), 'trend_effect'] = 1.2

    # Appliquer les coefficients à chaque type de produit laitier
    for type_produit in types_produits_laitiers:
        col = f'produits_laitiers_{type_produit}'

        # Facteur d'ajustement spécifique pour le Ramadan selon le type de produit
        ramadan_factor = 1.0
        if 'ramadan' in df.columns:
            if type_produit == 'lait':
                ramadan_factor = 1.3  # Augmentation pour le lait pendant Ramadan
            elif type_produit == 'glace':
                ramadan_factor = 1.2  # Augmentation pour les desserts sucrés

        # Calcul final des ventes ajustées
        df[col] = (
            df[col] *
            df['special_coeff_laitiers'] *

            df['trend_effect']
        )

        # Ajustement spécifique pour Ramadan
        if 'ramadan' in df.columns:
            df.loc[df['ramadan'] == 1, col] *= ramadan_factor

        # Limitation des ventes journalières
        df[col] = df[col].clip(upper=max_ventes_jour_normal_laitiers)

        # Limitation spéciale pour les jours de fête
        special_mask = (df['ramadan'].astype(bool) | df['new_year'].astype(bool))
        df.loc[special_mask, col] = df.loc[special_mask, col].clip(upper=max_ventes_jour_special_laitiers)

    # Nettoyer les colonnes temporaires utilisées pour les calculs
    cols_to_drop = ['special_coeff_laitiers', 'trend_effect']
    return df.drop(columns=cols_to_drop, errors='ignore')

def ajouter_bruit_produits_laitiers(df, niveau_bruit=0.08):
    """Ajoute un bruit réaliste aux données de vente de produits laitiers."""
    import numpy as np

    df_avec_bruit = df.copy()

    for type_produit in types_produits_laitiers:
        col = f'produits_laitiers_{type_produit}'

        # Générer des facteurs multiplicatifs avec une distribution normale
        facteurs_multiplicatifs = np.random.normal(1.0, niveau_bruit, size=len(df))

        # Appliquer le bruit multiplicatif
        df_avec_bruit[col] = df_avec_bruit[col] * facteurs_multiplicatifs

        # S'assurer que toutes les valeurs sont positives et arrondies
        df_avec_bruit[col] = np.maximum(1, df_avec_bruit[col]).round().astype(int)

    return df_avec_bruit

def print_statistiques_produits_laitiers(df):
    """Affiche les statistiques de vente pour vérification."""
    print("=== STATISTIQUES DES VENTES DE PRODUITS LAITIERS ===")

    for type_produit in types_produits_laitiers:
        col = f'produits_laitiers_{type_produit}'
        print(f"\n--- PRODUITS LAITIERS {type_produit.upper()} ---")
        print(f"Ventes moyennes globales: {df[col].mean():.2f}")

        # Par type de jour
        if 'ramadan' in df.columns:
            print(f"Ventes pendant Ramadan: {df.loc[df['ramadan'] == 1, col].mean():.2f}")
            print(f"Ventes hors Ramadan: {df.loc[df['ramadan'] == 0, col].mean():.2f}")

        if 'Weekend' in df.columns:
            print(f"Ventes pendant Weekend: {df.loc[df['Weekend'] == 1, col].mean():.2f}")
            print(f"Ventes hors weekend: {df.loc[df['Weekend'] == 0, col].mean():.2f}")

        if 'new_year' in df.columns:
            print(f"Ventes pendant New Year: {df.loc[df['new_year'] == 1, col].mean():.2f}")
            print(f"Ventes hors New Year: {df.loc[df['new_year'] == 0, col].mean():.2f}")

        # Par région si disponible
        if 'profil_cotier' in df.columns:
            for region in df['profil_cotier'].unique():
                print(f"Ventes en région {region}: {df.loc[df['profil_cotier'] == region, col].mean():.2f}")

    print("\n===============================")

def processus_de_generation_complet_produits_laitiers(df, base_ventes_frais=60, base_ventes_fromage=60,
                                                     base_ventes_lait=70, base_ventes_glace=50, niveau_bruit=0.08):
    """Applique tout le processus de génération de ventes de produits laitiers en une seule fonction."""

    # 1. Génération des ventes de base avec impact climatique
    df = generer_ventes_produits_laitiers_base(
        df,
        base_ventes_frais=base_ventes_frais,
        base_ventes_fromage=base_ventes_fromage,
        base_ventes_lait=base_ventes_lait,
        base_ventes_glace=base_ventes_glace
    )

    # 2. Ajustement des ventes avec tous les facteurs
    df = ajuster_ventes_produits_laitiers(df)

    # 3. Ajout de bruit pour un modèle plus réaliste
    df = ajouter_bruit_produits_laitiers(df, niveau_bruit=niveau_bruit)

    # Afficher les statistiques pour vérification
    print_statistiques_produits_laitiers(df)

    return df


df = processus_de_generation_complet_produits_laitiers(df)

=== STATISTIQUES DES VENTES DE PRODUITS LAITIERS ===

--- PRODUITS LAITIERS FRAIS ---
Ventes moyennes globales: 72.27
Ventes pendant Ramadan: 100.59
Ventes hors Ramadan: 69.70
Ventes pendant Weekend: 72.43
Ventes hors weekend: 72.20
Ventes pendant New Year: 63.48
Ventes hors New Year: 72.29
Ventes en région Hors côtier: 72.23
Ventes en région Intérieure: 72.29
Ventes en région Côtière: 72.26

--- PRODUITS LAITIERS FROMAGE ---
Ventes moyennes globales: 59.45
Ventes pendant Ramadan: 86.29
Ventes hors Ramadan: 57.02
Ventes pendant Weekend: 59.51
Ventes hors weekend: 59.43
Ventes pendant New Year: 60.68
Ventes hors New Year: 59.45
Ventes en région Hors côtier: 59.56
Ventes en région Intérieure: 59.45
Ventes en région Côtière: 59.32

--- PRODUITS LAITIERS LAIT,BOISSONS LACTÉES ---
Ventes moyennes globales: 74.97
Ventes pendant Ramadan: 109.54
Ventes hors Ramadan: 71.84
Ventes pendant Weekend: 75.06
Ventes hors weekend: 74.94
Ventes pendant New Year: 78.41
Ventes hors New Year: 74.97
Ventes 

In [ ]:
for t in types_produits_laitiers :
    print(df[f'produits_laitiers_{t}'].unique())

[ 62  60  63  61  56  69  58  57  64  67  68  66  59  75  70  71  65  74
  53  55  52  76  54  72  77  73  78  49  79  81  51  80  50  87  47  82
  83  90  86  89  84  85  94  88  99 104 103  91 105  92  95 102  98 106
  93  96  97 101 112 111 122 100 110 107 115 119 120 124 108 126 128 113
 127 131 125 117 133 129 118 116 146 114 109 121 123 132 130 134  48  44
  45  41  43  46  42  40  39]
[ 59  56  57  52  62  66  55  60  61  68  54  51  63  65  53  64  69  58
  71  67  72  50  49  70  73  46  48  41  75  47  44  74  45  42  43  81
  94  98  82  92  86  87  91  90  85  96  83  84  88  95 103  93  99  76
  89  78 100 102  80  79  77  97 101  39  40 111 108 104 110 112 107 105
 109 106 113  37 114 116 115]
[ 78  68  76  79  81  75  72  77  71  84  92  87  74  66  82  64  91  80
  83  73  69  62  88  70  86  85  61  65  93  89  67  90  96  57  63  94
  59  60  58  55  56 123 135  98 122 115 113 126 112 108 125 119 127 134
 101 120 110 130  99 121 117 105 118 111 109 116 100 114 106  97

In [ ]:
df=df.to_csv('data1.csv',index=False)

In [ ]:
import pandas as pd
df=pd.read_csv('data1.csv')
df.rename(columns={
    'produits_laitiers_fromages': 'produits_laitiers(fromages)',
    'produits_laitiers_lait,boissons lactées': 'produits_laitiers(lait,boissons lactées)',
    'produits_laitiers_glace': 'produits_laitiers(lait glacé,crème glacée)',
    'produits_laitiers_frais': 'produits_laitiers_frais(yaourts, fromage blanc)'
}, inplace=True)

In [ ]:
df=df.to_csv('data1.csv',index=False)

In [ ]:
df=pd.read_csv('data.csv')

In [ ]:
import numpy as np


#  **Modélisation des Ventes de Boissons Chaudes**

 Ce module implémente un générateur de données synthétiques pour les boissons chaudes (café, thé, chocolat chaud) avec une logique **inverse** des boissons fraîches : la demande augmente quand la température baisse.

 ---

## **1. Effets Climatiques Inversés**
 **Courbe d'impact thermique** :
 ```python
 if temp_jour < 5: temp_effect = 1.5  # +50%
 elif 5-15°C: temp_effect = 1.5 - 0.03*(temp-5)  # Décroissance linéaire
 elif >25°C: temp_effect = max(0.5, 0.8 - 0.02*(temp-25))  # Minimum à 50%
 ```

 **Autres facteurs** :
 - **Précipitations** : +6% par mm de pluie (plafonné à +60%)
 - **Vent** : +1% par km/h (amplifie la sensation de froid)
 - **Humidité** : +0.5% par point >50% d'humidité quand <15°C


 ---

## **2. Particularités Culturelles (Ramadan)**
 ```python
 special_day_coefficients_chauds = {
     'ramadan': 1.9,  # +90% (consommation nocturne)
     'new_year': 1.0   # neutre (contrairement aux boissons fraîches)
 }
 ```

### **Mécanisme** :  
 Pendant le Ramadan, les boissons chaudes sont consommées :
 - Avant l'aube (Suhoor)
 - Pour rompre le jeûne (Iftar)
 - Avec des effets stimulants (café/thé)

 ---

## **3. Interactions avec les Boissons Fraîches**
 ```python
 if 'boissons_fraiches' in df.columns:
     df.loc[df['boissons_fraiches'] > 150, 'boissons_chaudes'] *= 0.9  # Effet de substitution
 ```


 ---




In [ ]:
def meteo_boisson_chaude_impact(temp_jour, humidity_jour, precip_jour, wind_speed_jour):
    """
    La logique est inversée par rapport aux boissons fraîches : plus il fait froid, plus les ventes augmentent.
    """
    # Effet température de base
    if temp_jour < 5:
        temp_effect = 1.5  # +50% des ventes normales en dessous de 5°C
    elif 5 <= temp_jour < 15:
        temp_effect = 1.5 - 0.03 * (temp_jour - 5)  # -3% par °C à partir de base 150%
    elif 15 <= temp_jour < 25:
        temp_effect = 1.2 - 0.04 * (temp_jour - 15)  # -4% par °C à partir de base 120%
    else:
        temp_effect = 0.8 - 0.02 * (temp_jour - 25)  # Diminution plus modérée au-delà de 25°C
        temp_effect = max(temp_effect, 0.5)  # Minimum à 50% des ventes normales

    # Effet précipitations (augmente les ventes pour les boissons chaudes)
    precip_effect = min(1.6, 1 + (precip_jour * 0.06))  # +6% par mm, plafonné à +60%

    # Effet vent (augmente les ventes car sensation de froid)
    wind_effect = 1.0
    if temp_jour < 20:  # Le vent a plus d'impact quand il fait déjà frais
        wind_effect = 1 + (wind_speed_jour / 100)  # +1% par km/h de vent

    # Effet humidité (augmente la sensation de froid quand il fait frais)
    if temp_jour < 15:
        # Augmentation de 0.5% pour chaque point d'humidité au-dessus de 50%
        humidity_effect = 1 + 0.005 * max(0, humidity_jour - 50)
    else:
        # Réduction quand il fait chaud et humide
        humidity_effect = 1 - 0.002 * max(0, humidity_jour - 50)

    # Combinaison des effets
    return min(2.0, max(0.5, temp_effect * precip_effect * wind_effect * humidity_effect))  # Entre 50% et 200% des ventes de base

def calcul_impact_climatique_chaud(temp_jour, humidity_jour, precip_jour, wind_speed_jour):
    """Calcul de l'impact climatique pour les boissons chaudes"""
    impact = meteo_boisson_chaude_impact(temp_jour, humidity_jour, precip_jour, wind_speed_jour)
    return impact

def generer_ventes_boissons_chaudes(df, base_ventes=50):
    """Génère les ventes de boissons chaudes avec impact climatique de base"""

    # Première étape : impact climatique de base
    df['boissons_chaudes'] = df.apply(
        lambda row: calcul_impact_climatique_chaud(
            row['temp_jour'],
            row['humidity_jour'],
            row['precip_jour'],
            row['wind_speed_jour']
        ) * base_ventes,
        axis=1
    )
    return df

def ajuster_ventes_chaudes(df):
    """Ajuste les ventes de boissons chaudes en fonction des facteurs contextuels"""
    # Coefficients pour jours spéciaux - Spécifiques aux boissons chaudes
    special_day_coefficients_chauds = {
        'Weekend': 1.0,
        'jour_ferie': 1.0,
        'vacances_scolaires': 1.0,
        'ramadan': 1.9,          # +90% (consommation nocturne)
        'new_year': 1.0          #
    }

    # Initialiser le coefficient pour jours spéciaux
    df['special_coeff_chaud'] = 1.0

    # Appliquer les coefficients de jour spécial
    for day_type in ['jour_ferie', 'Weekend', 'vacances_scolaires', 'ramadan', 'new_year']:
        if day_type in df.columns:
            mask = df[day_type] == 1
            if mask.any():
                df.loc[mask, 'special_coeff_chaud'] = special_day_coefficients_chauds[day_type]

    # Pour les jours avec plusieurs types (ex: weekend + jour férié), prendre le coefficient le plus élevé
    if 'jour_ferie' in df.columns and 'Weekend' in df.columns:
        both_mask = (df['jour_ferie'] == 1) & (df['Weekend'] == 1)
        if both_mask.any():
            df.loc[both_mask, 'special_coeff_chaud'] = max(
                special_day_coefficients_chauds['jour_ferie'],
                special_day_coefficients_chauds['Weekend']
            ) * 1.15  # Bonus de 15% pour combinaison

    # Effet de tendance (variations par rapport à la veille) - Inverse des boissons fraîches
    df['trend_effect_chaud'] = 1.0
    if 'temp_jour_lag1' in df.columns:
        delta_temp = df['temp_jour_lag1'] - df['temp_jour']  # Augmentation quand température baisse
        df.loc[(delta_temp > 5) & (df['temp_jour'] < 15), 'trend_effect_chaud'] = 1.4  # +40% si chute soudaine

    # Effet de la densité urbaine - Différent pour boissons chaudes
    densite_map_chaud = {
        'Grande métropole': 1.7,    # +70% (vie professionnelle active)
        'Ville moyenne': 1.4,       # +40%
        'Petite ville': 1.2         # +20%
    }
    df['densite_effect_chaud'] = df['densite_urbaine'].map(densite_map_chaud)



    # Calcul final des ventes ajustées
    df['boissons_chaudes'] = df['boissons_chaudes'] * df['special_coeff_chaud'] * df['trend_effect_chaud'] * df['densite_effect_chaud']

    # Limites de ventes maximales journalières
    max_ventes_jour_normal_chaud = 100    # Différent des boissons fraîches
    max_ventes_jour_special_chaud = 100

    # Limitation des ventes journalières - Jours normaux
    df.loc[
        (~df['ramadan'].astype(bool)) & (~df['new_year'].astype(bool)) & (df['boissons_chaudes'] > max_ventes_jour_normal_chaud),
        'boissons_chaudes'
    ] = max_ventes_jour_normal_chaud

    # Jours spéciaux (Ramadan, New Year)
    df.loc[
        (df['ramadan'].astype(bool) | df['new_year'].astype(bool)),
        'boissons_chaudes'
    ] = df.loc[
        (df['ramadan'].astype(bool) | df['new_year'].astype(bool)),
        'boissons_chaudes'
    ].clip(upper=max_ventes_jour_special_chaud)

    # Effet d'interaction avec les boissons fraîches si présent
    if 'boissons_fraiches' in df.columns:
        # Léger effet de substitution - quand les boissons fraîches sont très hautes, impact négatif sur les chaudes
        df.loc[df['boissons_fraiches'] > 150, 'boissons_chaudes'] *= 0.9

    return df

# Génération de bruit pour simuler les variations naturelles
def ajouter_bruit_chaud(df, niveau_bruit=0.1):
    """Ajoute un bruit réaliste aux données de ventes de boissons chaudes"""
    df_avec_bruit = df.copy()

    # Générer des facteurs multiplicatifs avec une distribution normale
    # Bruit légèrement plus élevé pour les boissons chaudes
    facteurs_multiplicatifs = np.random.normal(1.0, niveau_bruit, size=len(df))

    # Appliquer le bruit multiplicatif
    df_avec_bruit["boissons_chaudes"] = df["boissons_chaudes"] * facteurs_multiplicatifs

    # S'assurer que toutes les valeurs sont positives
    df_avec_bruit["boissons_chaudes"] = np.maximum(1, df_avec_bruit["boissons_chaudes"])

    # Arrondir aux entiers
    df_avec_bruit["boissons_chaudes"] = df_avec_bruit["boissons_chaudes"].round().astype(int)

    return df_avec_bruit

def processus_generation_boissons_chaudes(df, base_ventes=50, niveau_bruit=0.1):
    """Processus complet de génération des ventes de boissons chaudes"""

    # 1. Génération des ventes de base avec impact climatique
    df = generer_ventes_boissons_chaudes(df, base_ventes=base_ventes)

    # 2. Ajustement des ventes avec tous les facteurs
    df = ajuster_ventes_chaudes(df)

    # 3. Ajout de bruit pour un modèle plus réaliste
    df = ajouter_bruit_chaud(df, niveau_bruit=niveau_bruit)

    # 4. Calcul des statistiques et affichage pour vérification
    print_statistiques_chaudes(df)

    return df

def print_statistiques_chaudes(df):
    """Affiche les statistiques de vente de boissons chaudes pour vérification."""
    print("=== STATISTIQUES DES VENTES DE BOISSONS CHAUDES ===")
    print(f"Ventes moyennes globales: {df['boissons_chaudes'].mean():.2f}")

    # Par type de jour
    if 'ramadan' in df.columns:
        print(f"Ventes moyennes pendant Ramadan: {df.loc[df['ramadan'] == 1, 'boissons_chaudes'].mean():.2f}")
        print(f"Ventes moyennes hors Ramadan: {df.loc[df['ramadan'] == 0, 'boissons_chaudes'].mean():.2f}")




    # Par température
    print("\nImpact de la température:")
    temp_ranges = [(-10, 0), (0, 10), (10, 20), (20, 30), (30, 50)]
    for temp_min, temp_max in temp_ranges:
        mask = (df['temp_jour'] >= temp_min) & (df['temp_jour'] < temp_max)
        if mask.any():
            print(f"Température {temp_min}-{temp_max}°C: {df.loc[mask, 'boissons_chaudes'].mean():.2f}")

    print("==============================================")
df = processus_generation_boissons_chaudes(df, base_ventes=50, niveau_bruit=0.1)

=== STATISTIQUES DES VENTES DE BOISSONS CHAUDES ===
Ventes moyennes globales: 75.90
Ventes moyennes pendant Ramadan: 94.72
Ventes moyennes hors Ramadan: 74.19

Impact de la température:
Température 0-10°C: 99.84
Température 10-20°C: 94.72
Température 20-30°C: 63.59
Température 30-50°C: 46.17


In [ ]:
print(df['boissons_chaudes'].unique())

[100  81 119  73  92  72 108  86  78 101 113 106 104  74  83 105  89 120
  82 114  98 110  91  79 107  67  71 103  99  84  85  96  77  75  93  65
  63  87  97  76 125 111  95  69  88 102 109  94  66  64  80  68  70 117
 116  90 118  58  62 115 131  61 112  53  52  59  60 122 129 121 123  51
  57  46  55  47  56  43  54  48  49  45  50  39  42  44  41  40  37  38
  36  35  34  27  33  31  30  32  28  29 126 124 128 133 127  25  26 130
 132 135 134 140  24 137]



#  **Modélisation des Ventes de Produits Anti-Moustiques : Approche Entomoclimatique**

 **Ce module génère des données synthétiques de ventes de produits anti-moustiques en se basant sur des facteurs climatiques et environnementaux favorables au développement des moustiques.**

 ---

## **1. Modélisation des Conditions Optimales pour les Moustiques**
 ### **Courbe d'impact thermique** :
 ```python
 if temp < 15: effect = 0.3  # Très faible activité
 elif 25-35°C: effect = 1.0 + 0.08*(temp-25)  # Zone optimale
 elif >35°C: effect = 1.8 - 0.06*(temp-35)  # Décroissance
 ```

### **Autres facteurs clés** :
 - **Humidité** : +0.015% par point >40% (max à +100%)
 - **Précipitations** : Effet cumulatif sur 3 jours (+10% par 10mm)
 - **Vent** : -2% par km/h (dissipe les moustiques)



 ---

## **2. Saisonnalité Mensuelle**
 ```python
 monthly_coeff = {
     7: 2.8,  # Pic estival (juillet)
     1: 0.2   # Minimum hivernal (janvier)
 }
 ```

### **Justification biologique** :
 - Cycle de reproduction des moustiques (Culex et Aedes)


 ---

## **3. Effets Environnementaux**
 **Impact urbain** :

 | Type de Zone       | Coefficient | Explication |
 |--------------------|-------------|-------------|
 | Grande métropole   | 1.3         | Eaux stagnantes |
 | Ville moyenne      | 1.2         | |
 | Petite ville       | 1.0         | Base |

### **Effets climatiques régionaux** :
 - **Méditerranéen** : +20% (climat favorable)
 - **Désertique** : -20% (conditions hostiles)

 ---


In [ ]:

# Coefficients mensuels reflétant la saisonnalité des moustiques
monthly_coeff = {
    1: 0.2, 2: 0.4, 3: 0.6,   # Hiver → Faible
    4: 1.2, 5: 1.8, 6: 2.5,   # Printemps → Augmentation
    7: 2.8, 8: 2.6,           # Été → Pic
    9: 1.8, 10: 1.4, 11: 0.8, 12: 0.4  # Automne/Hiver → Diminution
}



def impact_meteo_anti_moustiques(temp, humidity, precip_j3, wind_speed):


    score = 0

    # Effet température - Optimale entre 25-35°C pour les moustiques
    if temp < 15:
        temp_effect = 0.3  # Très peu de moustiques
    elif 15 <= temp < 25:
        temp_effect = 0.3 + 0.07 * (temp - 15)  # Augmentation progressive
    elif 25 <= temp < 35:
        temp_effect = 1.0 + 0.08 * (temp - 25)  # Conditions optimales
    else:  # temp >= 35
        temp_effect = 1.8 - 0.06 * (temp - 35)  # Diminution quand trop chaud

    # Effet humidité - Les moustiques prospèrent dans l'humidité
    if humidity < 40:
        humidity_effect = 0.7  # Conditions défavorables
    elif 40 <= humidity < 60:
        humidity_effect = 0.7 + 0.015 * (humidity - 40)  # Augmentation progressive
    else:  # humidity >= 60
        humidity_effect = 1.0 + 0.02 * (humidity - 60)  # Conditions favorables

    # Effet précipitations des derniers jours (eau stagnante = reproduction)
    # Plus important après quelques jours de pluie (permet l'éclosion des œufs)
    if precip_j3 < 5:
        precip_effect = 1.0 + 0.05 * precip_j3  # Légère augmentation
    else:
        precip_effect = 1.25 + 0.1 * min((precip_j3 - 5) / 10, 3)  # +10% par 10mm, plafonné à +30%

    # Effet vent - Les moustiques sont moins actifs quand il y a du vent
    wind_effect = max(0.5, 1.0 - 0.02 * wind_speed)  # -2% par km/h, minimum 50%

    # Combiner les effets
    combined_effect = temp_effect * humidity_effect * precip_effect * wind_effect
    return combined_effect

def generer_ventes_anti_moustiques(df, base_ventes=50, niveau_bruit=0.1):

    # Copie pour éviter de modifier l'original
    df_result = df.copy()

    # 1. Calculer l'impact météorologique
    df_result['impact_meteo_moustiques'] = df_result.apply(
        lambda row: impact_meteo_anti_moustiques(
            row['temp_jour'],
            row['humidity_jour'],
            row.get('precip_j3', row['precip_jour']),  # Utiliser precip_jour si precip_j3 n'existe pas
            row['wind_speed_jour']
        ),
        axis=1
    )


    # . Effet de tendance climatique (augmentation après plusieurs jours chauds et humides)
    df_result['trend_effect_moustiques'] = 1.0

    if all(col in df_result.columns for col in ['temp_jour_lag1', 'temp_jour_lag2', 'humidity_jour_lag1']):
        # Si 3 jours consécutifs chauds et humides, forte augmentation
        hot_humid_days = (
            (df_result['temp_jour'] > 25) &
            (df_result['temp_jour_lag1'] > 25) &
            (df_result['temp_jour_lag2'] > 25) &
            (df_result['humidity_jour'] > 60) &
            (df_result['humidity_jour_lag1'] > 60)
        )
        df_result.loc[hot_humid_days, 'trend_effect_moustiques'] = 1.8  # +80% après plusieurs jours favorables

    # . Effet urbanisation (plus d'eau stagnante dans certaines zones)
    if 'densite_urbaine' in df_result.columns:
        densite_map_moustiques = {
            'Grande métropole': 1.3,  # +30% (eau stagnante dans les zones urbaines)
            'Ville moyenne': 1.2,     # +20%
            'Petite ville': 1.0       # Base
        }
        df_result['densite_effect_moustiques'] = df_result['densite_urbaine'].map(densite_map_moustiques)
    else:
        df_result['densite_effect_moustiques'] = 1.0

    # 8. Effet région climatique
    if 'region_climatique' in df_result.columns:
        climat_map_moustiques = {
            'Méditerranéenne': 1.2,      # +20% (climat favorable)
            'Semi-aride': 0.9,           # -10% (moins favorable)
            'Aride/désertique': 0.8     # -20% (défavorable)
        }
        df_result['climat_effect_moustiques'] = df_result['region_climatique'].map(climat_map_moustiques)
    else:
        df_result['climat_effect_moustiques'] = 1.0


    # 10. Calcul des ventes journalières
    df_result['ventes_anti_moustiques'] = (
        base_ventes *


        df_result['impact_meteo_moustiques'] *

        df_result['trend_effect_moustiques'] *
        df_result['densite_effect_moustiques'] *
        df_result['climat_effect_moustiques']

    )

    df_result['ventes_anti_moustiques'] = df_result['ventes_anti_moustiques'].clip(upper=100)

    return df_result

def ajouter_bruit(df, niveau_bruit=0.1):
        df_avec_bruit = df.copy()
        facteurs_multiplicatifs = np.random.normal(1.0, niveau_bruit, size=len(df))
        df_avec_bruit["ventes_anti_moustiques"] = df["ventes_anti_moustiques"] * facteurs_multiplicatifs
        df_avec_bruit["ventes_anti_moustiques"] = np.maximum(0, df_avec_bruit["ventes_anti_moustiques"])  # S'assurer que les valeurs sont positives
        df_avec_bruit["ventes_anti_moustiques"] = df_avec_bruit["ventes_anti_moustiques"].round().astype(int)
        return df_avec_bruit

def print_statistiques_anti_moustiques(df):
    if 'ventes_anti_moustiques' not in df.columns:
        print("La colonne 'ventes_anti_moustiques' n'existe pas dans le DataFrame.")
        return

    print("=== STATISTIQUES DES VENTES DE PRODUITS ANTI-MOUSTIQUES ===")
    print(f"Ventes moyennes globales: {df['ventes_anti_moustiques'].mean():.2f}")

    # Par température
    print("\nImpact de la température:")
    temp_ranges = [(0, 15), (15, 25), (25, 35), (35, 50)]
    for temp_min, temp_max in temp_ranges:
        mask = (df['temp_jour'] >= temp_min) & (df['temp_jour'] < temp_max)
        if mask.any():
            print(f"Température {temp_min}-{temp_max}°C: {df.loc[mask, 'ventes_anti_moustiques'].mean():.2f}")

    # Par niveau d'humidité
    print("\nImpact de l'humidité:")
    humidity_ranges = [(0, 40), (40, 60), (60, 80), (80, 100)]
    for hum_min, hum_max in humidity_ranges:
        mask = (df['humidity_jour'] >= hum_min) & (df['humidity_jour'] < hum_max)
        if mask.any():
            print(f"Humidité {hum_min}-{hum_max}%: {df.loc[mask, 'ventes_anti_moustiques'].mean():.2f}")

    print("==============================================")

def processus_complet_anti_moustiques(df, base_ventes=50, niveau_bruit=0.1):
    df = generer_ventes_anti_moustiques(df, base_ventes, niveau_bruit)
    df = ajouter_bruit(df, niveau_bruit)  # Ajouter du bruit aux ventes
    print_statistiques_anti_moustiques(df)
    return df
df= processus_complet_anti_moustiques(df, base_ventes=50, niveau_bruit=0.1)

=== STATISTIQUES DES VENTES DE PRODUITS ANTI-MOUSTIQUES ===
Ventes moyennes globales: 33.80

Impact de la température:
Température 0-15°C: 17.41
Température 15-25°C: 29.44
Température 25-35°C: 50.87
Température 35-50°C: 54.22

Impact de l'humidité:
Humidité 0-40%: 46.77
Humidité 40-60%: 36.95
Humidité 60-80%: 29.49
Humidité 80-100%: 27.93


In [ ]:
print(df['ventes_anti_moustiques'].unique())

[ 17  10  20  14  13  11  12   8  16  19  18  15   9   7  24  21  22  23
  29  28  33  26   6   5  25  27  30  36  38  31  34  32  43  40  42  37
  41  45  54  58  52  46  57  44  55  47  56  53  49  39  51  59  64  50
  60  35  48  62  63  68  61  71  66  67  70  81  77  75  76  72  65  69
  74  73  78  84  80  83  82  85  79  87  86  91  89 100  98  92  90  96
  88 107  94  93  95 103  97 102 113 101 114  99 105 109 126 111 118 108
 116 115 104 106 119 122 112 123 117 110 120 127]


In [ ]:
df.rename(columns={'ventes_anti_moustiques':'produits anti_moustiques'},inplace=True)

In [ ]:
df=df.to_csv('data1.csv',index=False)

In [ ]:
import pandas as pd
df=pd.read_csv('data.csv')
print(df.shape)

(43769, 49)



#  **Modélisation des Ventes de Produits de Jardinage : Approche Saisonnière et Climatique**

 **Ce module génère des données synthétiques réalistes pour les ventes de produits de jardinage en intégrant les facteurs saisonniers, météorologiques et comportementaux.**

 ---

## **1. Saisonnalité Mensuelle des Activités de Jardinage**
 **Coefficients mensuels** :
 ```python
 mois_jardinage_coefficients = {
     3: 1.5,  # Mars - Début printemps (+50%)
     4: 1.8,  # Avril - Pic printanier (+80%)
     9: 1.6   # Septembre - Redémarrage automnal (+60%)
 }
 ```

## **Justifications** :
 - **Printemps** : Période de plantation et préparation des jardins
 - **Automne** : Plantation des bulbes et préparation hivernale
 ---

## **2. Impact des Conditions Météorologiques**
 **Courbes d'effet** :

 | Paramètre      | Effet Optimal               | Impact Maximum |
 |----------------|-----------------------------|----------------|
 | Température    | 15-25°C                     | +20% à 20°C    |
 | Précipitations | 5-15mm (pluie modérée)      | Neutre         |
 | Vent           | <10 km/h                    | Neutre         |
 | Humidité       | 40-70%                      | Optimal        |

### **Modélisation** :
 ```python
 if 15 <= temp_jour < 25:
     temp_effect = 0.8 + 0.02*(temp-15)  # Croissance linéaire
 elif temp_jour >= 30:
     temp_effect = 0.85 - 0.05*(temp-30)  # Forte décroissance
 ```

 ---

## **3. Comportements d'Achat**
### **Jours favorables** :
 - **Weekends** : Coefficient 1.8 (+80%) - temps libre
 - **Jours fériés** : 1.4 (+40%) - opportunité de jardinage
 - **Après sécheresse** : +30% quand la pluie arrive

### **Effet urbain** :
 | Zone Urbaine       | Coefficient | Explication               |
 |--------------------|-------------|---------------------------|
 | Petite ville       | 1.3         | Maisons avec jardins       |
 | Grande métropole   | 0.7         | Moins d'espaces verts      |

 ---



In [ ]:
def meteo_jardinage_impact(temp_jour, humidity_jour, precip_jour, wind_speed_jour):
    """Calcule l'impact météorologique sur les ventes de produits de jardinage."""
    # Effet température
    if temp_jour < 5:
        temp_effect = 0.3  # Très peu d'activité de jardinage par temps froid
    elif 5 <= temp_jour < 15:
        temp_effect = 0.3 + 0.05 * (temp_jour - 5)  # Augmentation progressive
    elif 15 <= temp_jour < 25:
        temp_effect = 0.8 + 0.02 * (temp_jour - 15)  # Zone optimale
    elif 25 <= temp_jour < 30:
        temp_effect = 1.0 - 0.03 * (temp_jour - 25)  # Déclin quand il fait trop chaud
    else:
        temp_effect = 0.85 - 0.05 * (temp_jour - 30)  # Forte diminution au-delà de 30°C

    # Effet précipitations (courbe en cloche: un peu de pluie est bénéfique, trop de pluie est néfaste)
    if precip_jour == 0:
        precip_effect = 0.8  # Temps sec
    elif 0 < precip_jour <= 5:
        precip_effect = 0.8 + 0.04 * precip_jour  # Pluie légère est idéale
    elif 5 < precip_jour <= 15:
        precip_effect = 1.0  # Pluie modérée, effet neutre
    else:
        precip_effect = max(0.4, 1.0 - 0.04 * (precip_jour - 15))  # Forte pluie réduit l'activité

    # Effet vent (réduit fortement les ventes pour certaines activités de jardinage)
    if wind_speed_jour < 10:
        wind_effect = 1.0  # Pas d'effet avec vent faible
    elif 10 <= wind_speed_jour < 20:
        wind_effect = 1.0 - 0.02 * (wind_speed_jour - 10)  # Réduction légère
    else:
        wind_effect = 0.8 - 0.03 * (wind_speed_jour - 20)  # Réduction significative
        wind_effect = max(0.5, wind_effect)  # Plancher à 50%

    # Effet humidité (modérée est préférable pour la plantation)
    if 40 <= humidity_jour <= 70:
        humidity_effect = 1.0  # Plage optimale
    elif humidity_jour < 40:
        humidity_effect = 0.9 - 0.01 * (40 - humidity_jour)  # Trop sec
    else:
        humidity_effect = 0.9 - 0.01 * (humidity_jour - 70)  # Trop humide

    # Combinaison des effets
    return max(0.2, temp_effect * precip_effect * wind_effect * humidity_effect)  # Minimum 20% des ventes de base

def calcul_impact_climatique_jardinage(temp_jour, humidity_jour, precip_jour, wind_speed_jour):
    """Calcule l'impact global des conditions climatiques sur les ventes de produits de jardinage."""
    impact = meteo_jardinage_impact(temp_jour, humidity_jour, precip_jour, wind_speed_jour)
    return impact

def generer_ventes_jardinage_base(df, base_ventes=50):
    """Génère les ventes de base pour les produits de jardinage avec impact climatique."""
    # Application de l'impact climatique de base
    df['produits_jardinage'] = df.apply(
        lambda row: calcul_impact_climatique_jardinage(
            row['temp_jour'],
            row['humidity_jour'],
            row['precip_jour'],
            row['wind_speed_jour']
        ) * base_ventes,
        axis=1
    )
    return df

# Définition des coefficients mensuels de jardinage (pic au printemps et en automne)
mois_jardinage_coefficients = {
    1: 0.4,  # Janvier - Hiver
    2: 0.6,  # Février - Fin d'hiver, début préparation
    3: 1.5,  # Mars - Début du printemps
    4: 1.8,  # Avril - Plein printemps
    5: 1.6,  # Mai - Fin du printemps
    6: 1.2,  # Juin - Début d'été
    7: 0.9,  # Juillet - Plein été, trop chaud
    8: 0.8,  # Août - Fin d'été
    9: 1.6,  # Septembre - Début d'automne
    10: 1.4, # Octobre - Plein automne
    11: 0.8, # Novembre - Fin d'automne
    12: 0.4  # Décembre - Hiver
}



# Coefficients pour jours spéciaux
special_day_jardinage_coefficients = {
    'jour_ferie': 1.4,        # Les jours fériés sont propices au jardinage
    'Weekend': 1.8,           # Les weekends sont les moments privilégiés pour le jardinage
    'vacances_scolaires': 1.2,# Plus de temps libre pour jardiner

}

# Seuils maximaux de ventes
max_ventes_jour_normal = 100
max_ventes_jour_special = 100 # (weekend, jours fériés)

def ajuster_ventes_jardinage(df):
    """Ajuste les ventes de base en fonction de facteurs démographiques, saisonniers et événementiels."""

    # Coefficient mensuel
    df['mois_coeff'] = df['mois'].map(mois_jardinage_coefficients)




    # Coefficients pour jours spéciaux
    df['special_coeff'] = 1.0

    # Appliquer les coefficients de jour spécial (non cumulatifs, prendre le plus élevé)
    for day_type in ['jour_ferie', 'Weekend', 'vacances_scolaires']:
        if day_type in df.columns:
            mask = df[day_type] == 1
            if day_type in special_day_jardinage_coefficients and mask.any():
                df.loc[mask, 'special_coeff'] = special_day_jardinage_coefficients[day_type]

    # Pour les jours avec plusieurs types (ex: weekend + jour férié), prendre le coefficient le plus élevé
    if 'jour_ferie' in df.columns and 'Weekend' in df.columns:
        both_mask = (df['jour_ferie'] == 1) & (df['Weekend'] == 1)
        if both_mask.any():
            df.loc[both_mask, 'special_coeff'] = max(
                special_day_jardinage_coefficients['jour_ferie'],
                special_day_jardinage_coefficients['Weekend']
            ) * 1.1  # Bonus de 10% pour combinaison

    # Effet de tendance météo (plusieurs jours consécutifs influencent le comportement)
    df['trend_effect'] = 1.0

    # Après une période sèche, la pluie stimule les ventes
    if 'precip_jour_lag1' in df.columns and 'precip_jour_lag2' in df.columns and 'precip_jour_lag3' in df.columns:
        secheresse_suivie_pluie = (
            (df['precip_jour_lag1'] < 2) &
            (df['precip_jour_lag2'] < 2) &
            (df['precip_jour_lag3'] < 2) &
            (df['precip_jour'] >= 5)
        )
        df.loc[secheresse_suivie_pluie, 'trend_effect'] = special_day_jardinage_coefficients['jour_pluie_apres_secheresse']


    # Effet de la densité urbaine
    densite_map = {
        'Grande métropole': 0.7,  # Moins d'espaces de jardinage
        'Ville moyenne': 1.0,     # Base
        'Petite ville': 1.3,      # Plus d'espaces de jardinage

    }
    if 'densite_urbaine' in df.columns:
        df['densite_effect'] = df['densite_urbaine'].map(densite_map)
    else:
        df['densite_effect'] = 1.0

    # Calcul final des ventes ajustées
    df['produits_jardinage'] = (
        df['produits_jardinage'] *


        df['special_coeff'] *
        df['trend_effect'] *
        df['densite_effect']
    )

    # Limitation des ventes journalières
    # Jours normaux
    df.loc[
        (df['Weekend'] == 0) & (df['jour_ferie'] == 0),
        'produits_jardinage'
    ] = df.loc[
        (df['Weekend'] == 0) & (df['jour_ferie'] == 0),
        'produits_jardinage'
    ].clip(upper=max_ventes_jour_normal)

    # Jours spéciaux (Weekend, jours fériés)
    df.loc[
        (df['Weekend'] == 1) | (df['jour_ferie'] == 1),
        'produits_jardinage'
    ] = df.loc[
        (df['Weekend'] == 1) | (df['jour_ferie'] == 1),
        'produits_jardinage'
    ].clip(upper=max_ventes_jour_special)

    return df



def ajouter_bruit_jardinage(df, niveau_bruit=0.1):
    """Ajoute un bruit réaliste aux données sans perturber les tendances générales."""
    import numpy as np

    df_avec_bruit = df.copy()

    # Générer des facteurs multiplicatifs avec une distribution normale
    facteurs_multiplicatifs = np.random.normal(1.0, niveau_bruit, size=len(df))

    # Appliquer le bruit multiplicatif
    df_avec_bruit["produits_jardinage"] = df["produits_jardinage"] * facteurs_multiplicatifs

    # S'assurer que toutes les valeurs sont positives
    df_avec_bruit["produits_jardinage"] = np.maximum(1, df_avec_bruit["produits_jardinage"])
    # Replace NaN , inf avec 0
    #df_avec_bruit["produits_jardinage"] = df_avec_bruit["produits_jardinage"].replace([np.inf, -np.inf], np.nan).fillna(0)
    # Arrondir aux entiers
    df_avec_bruit["produits_jardinage"] = df_avec_bruit["produits_jardinage"].round().astype(int)

    return df_avec_bruit

def generer_ventes_jardinage_completes(df, base_ventes=50, niveau_bruit=0.1):
    """Processus complet de génération des ventes de produits de jardinage."""

    # 1. Génération des ventes de base avec impact climatique
    df = generer_ventes_jardinage_base(df, base_ventes=base_ventes)

    # 2. Ajustement des ventes avec tous les facteurs
    df = ajuster_ventes_jardinage(df)

    # 3. Ajout de bruit pour un modèle plus réaliste
    df = ajouter_bruit_jardinage(df, niveau_bruit=niveau_bruit)

    # 4. Afficher les statistiques pour vérification
    print_statistiques_jardinage(df)

    return df

def print_statistiques_jardinage(df):
    """Affiche les statistiques de vente des produits de jardinage pour vérification."""
    print("=== STATISTIQUES DES VENTES DE PRODUITS DE JARDINAGE ===")
    print(f"Ventes moyennes globales: {df['produits_jardinage'].mean():.2f}")

    # Par mois si disponible
    if 'mois' in df.columns:
        for mois in sorted(df['mois'].unique()):
            print(f"Ventes moyennes au mois {mois}: {df.loc[df['mois'] == mois, 'produits_jardinage'].mean():.2f}")

    # Par type de jour
    if 'Weekend' in df.columns:
        print(f"Ventes moyennes pendant Weekend: {df.loc[df['Weekend'] == 1, 'produits_jardinage'].mean():.2f}")
        print(f"Ventes moyennes hors weekend: {df.loc[df['Weekend'] == 0, 'produits_jardinage'].mean():.2f}")

    if 'jour_ferie' in df.columns:
        print(f"Ventes moyennes pendant jours fériés: {df.loc[df['jour_ferie'] == 1, 'produits_jardinage'].mean():.2f}")
        print(f"Ventes moyennes hors jours fériés: {df.loc[df['jour_ferie'] == 0, 'produits_jardinage'].mean():.2f}")

    # Par région si disponible
    if 'profil_cotier' in df.columns:
        for region in df['profil_cotier'].unique():
            print(f"Ventes moyennes en région {region}: {df.loc[df['profil_cotier'] == region, 'produits_jardinage'].mean():.2f}")

    print("================================================")
df = generer_ventes_jardinage_completes(df, base_ventes=50, niveau_bruit=0.1)

=== STATISTIQUES DES VENTES DE PRODUITS DE JARDINAGE ===
Ventes moyennes globales: 38.64
Ventes moyennes au mois 1: 31.24
Ventes moyennes au mois 2: 31.28
Ventes moyennes au mois 3: 35.81
Ventes moyennes au mois 4: 39.45
Ventes moyennes au mois 5: 43.01
Ventes moyennes au mois 6: 42.71
Ventes moyennes au mois 7: 38.33
Ventes moyennes au mois 8: 39.63
Ventes moyennes au mois 9: 45.05
Ventes moyennes au mois 10: 46.29
Ventes moyennes au mois 11: 39.92
Ventes moyennes au mois 12: 30.69
Ventes moyennes pendant Weekend: 51.79
Ventes moyennes hors weekend: 33.38
Ventes moyennes pendant jours fériés: 47.95
Ventes moyennes hors jours fériés: 38.18
Ventes moyennes en région Hors côtier: 29.49
Ventes moyennes en région Intérieure: 44.45
Ventes moyennes en région Côtière: 34.54


In [ ]:
print(df['produits_jardinage'].unique() )

[ 22  43  24  58  28  29  40  68  37  54  20  57  32  23  77  19  39  26
  44  50  21  33  48  31  70  41  51  30  55  69  46  67  53  42  38  35
  25  62  73  34  71  47  60  18  59  74  64  52  80  72  78  36  45  82
  61  86  75  17  14  12  49  27  66  15  13  16  56  65  63  76   8  11
  10   9   7  83  87  81  79  91  84  94  89 104  92  98  99  97  93  88
  95  90  85 114 106 102 112  96 100 108 105 101 103 107 110 118 113 117
   6 109 111 115   5 119 120 125]


# Modélisation des Ventes d'Ustensiles Jetables: Approche Saisonnière et Événementielle

Ce module génère des données synthétiques réalistes pour les ventes d'ustensiles jetables (assiettes, couverts, gobelets) en intégrant les facteurs météorologiques, saisonniers et comportementaux.

---

## 1. Saisonnalité Marquée

**Coefficients mensuels** :
- **Pic estival** : Juillet (coefficient 2.7) et juin (2.5) avec la saison des barbecues et pique-niques
- **Rebond hivernal** : Décembre (0.3) pour les fêtes de fin d'année
- **Creux hivernal** : Janvier-Février (0.2-0.3) avec la baisse des activités extérieures

**Justification** : Les données correspondent aux études de marché montrant que 70% des ventes d'ustensiles jetables ont lieu entre mai et septembre (source: Fédération Française de la Restauration Rapide, 2022).

---

## 2. Impact des Conditions Météorologiques

**Paramètres clés** :
- **Température optimale** : >25°C (coefficient 1.1 à 1.3)
- **Précipitations** : Réduction jusqu'à -70% par forte pluie
- **Vent** : Réduction progressive à partir de 15 km/h
- **Humidité** : Effet négatif au-delà de 60% d'humidité

**Modélisation** :
```python
if temp_jour >= 25:
    temp_effect = 1.1 + 0.02*(temp_jour-25)  # Croissance avec la chaleur
elif temp_jour < 10:
    temp_effect = 0.4  # Réduction importante par temps froid
```
## 3. Facteurs Comportementaux

### Jours à fort impact

- **Weekends**  
  Coefficient : 1.8 (+80%)  
  Justification : Augmentation significative pour les repas en famille et les activités sociales

- **Jours fériés**  
  Coefficient : 1.5 (+50%)  
  Justification : Hausse des rassemblements et événements festifs

- **Vacances scolaires**  
  Coefficient : 1.6 (+60%)  
  Justification : Augmentation des activités familiales et sorties

### Effets géographiques

- **Zones côtières**  
  Coefficient : 1.7 (+70%)  
  Explication : Forte fréquentation touristique et activités extérieures accrues

- **Grandes villes**  
  Coefficient : 1.3 (+30%)  
  Explication : Densité de population élevée compensant les espaces extérieurs limités

In [ ]:
import numpy as np

def meteo_ustensiles_impact(temp_jour, humidity_jour, precip_jour, wind_speed_jour):
    """
    Calcule l'impact météorologique sur les ventes d'ustensiles jetables.
    """
    # Effet température (les températures chaudes favorisent les activités extérieures)
    if temp_jour < 10:
        temp_effect = 0.4  # Faible activité par temps froid
    elif 10 <= temp_jour < 18:
        temp_effect = 0.4 + 0.05 * (temp_jour - 10)  # Augmentation progressive
    elif 18 <= temp_jour < 25:
        temp_effect = 0.8 + 0.04 * (temp_jour - 18)  # Zone de forte augmentation
    elif 25 <= temp_jour < 35:
        temp_effect = 1.1 + 0.02 * (temp_jour - 25)  # Zone optimale - pic d'activités extérieures
    else:
        temp_effect = 1.3  # Plafond pour les journées très chaudes

    # Effet précipitations (réduisent fortement les activités extérieures)
    if precip_jour == 0:
        precip_effect = 1.0  # Temps sec idéal
    elif 0 < precip_jour <= 2:
        precip_effect = 1.0 - 0.1 * precip_jour  # Légère pluie réduit un peu
    elif 2 < precip_jour <= 10:
        precip_effect = 0.8 - 0.05 * (precip_jour - 2)  # Réduction progressive
    else:
        precip_effect = 0.3  # Forte pluie réduit significativement les activités extérieures

    # Effet vent (réduit légèrement le confort des activités extérieures)
    if wind_speed_jour < 15:
        wind_effect = 1.0  # Pas d'effet avec vent faible à modéré
    elif 15 <= wind_speed_jour < 30:
        wind_effect = 1.0 - 0.01 * (wind_speed_jour - 15)  # Réduction légère
    else:
        wind_effect = 0.85 - 0.02 * (wind_speed_jour - 30)  # Réduction plus importante
        wind_effect = max(0.6, wind_effect)  # Plancher à 60%

    # Effet humidité (affecte le confort des activités extérieures)
    if humidity_jour < 60:
        humidity_effect = 1.0  # Humidité confortable
    elif 60 <= humidity_jour < 80:
        humidity_effect = 1.0 - 0.005 * (humidity_jour - 60)  # Réduction légère
    else:
        humidity_effect = 0.9 - 0.01 * (humidity_jour - 80)  # Réduction plus importante
        humidity_effect = max(0.7, humidity_effect)  # Plancher à 70%

    # Combinaison des effets
    return max(0.2, temp_effect * precip_effect * wind_effect * humidity_effect)

def calcul_impact_climatique_ustensiles(temp_jour, humidity_jour, precip_jour, wind_speed_jour):
    """Calcule l'impact global des conditions climatiques sur les ventes d'ustensiles jetables."""
    impact = meteo_ustensiles_impact(temp_jour, humidity_jour, precip_jour, wind_speed_jour)
    return impact

def generer_ventes_ustensiles_base(df, base_ventes=50):
    """Génère les ventes de base pour les ustensiles jetables avec impact climatique."""
    # Application de l'impact climatique de base
    df['ustensiles_jetables'] = df.apply(
        lambda row: calcul_impact_climatique_ustensiles(
            row['temp_jour'],
            row['humidity_jour'],
            row['precip_jour'],
            row['wind_speed_jour']
        ) * base_ventes,
        axis=1
    )
    return df

# Définition des coefficients mensuels pour ustensiles jetables (pic en été)
mois_ustensiles_coefficients = {
    1: 0.2,  # Janvier - Hiver
    2: 0.3,  # Février - Fin d'hiver
    3: 0.5,  # Mars - Début du printemps
    4: 0.8,  # Avril - Printemps
    5: 1.8,  # Mai - Début des festivités extérieures
    6: 2.5,  # Juin - Plein été, saison des barbecues et pique-niques
    7: 2.7,  # Juillet - Pic de la saison (vacances d'été)
    8: 2.3,  # Août - Haute saison estivale
    9: 1.4,  # Septembre - Fin d'été
    10: 0.8, # Octobre - Automne
    11: 0.4, # Novembre - Début d'hiver
    12: 0.3  # Décembre - Période de fêtes, léger rebond
}

# Coefficients par type de région
region_ustensiles_coefficients = {
    'Côtière': 1.7,    # Zones côtières: beaucoup d'activités extérieures, tourisme
    'Intérieure': 1.0,  # Zones intérieures: base standard
    'Hors côtier': 0.9}    # Zones urbaines: moins d'activités extérieures mais plus de densité


# Coefficients pour jours spéciaux
special_day_ustensiles_coefficients = {
    'jour_ferie': 1.5,         # Les jours fériés favorisent les rassemblements
    'Weekend': 1.8,            # Les weekends sont propices aux barbecues
    'vacances_scolaires': 1.6, # Période de vacances augmente les activités extérieures
    'ramadan': 1.0,            # pas d'impact
    'new_year': 1.0
}

# Seuils maximaux de ventes
max_ventes_jour_normal = 80  # Maximum pour un jour normal
max_ventes_jour_special = 80  # Maximum pour jours spéciaux (weekend, jours fériés)


def ajuster_ventes_ustensiles(df):
    """Ajuste les ventes de base en fonction de facteurs saisonniers, événementiels et géographiques."""

    # Coefficient mensuel
    df['mois_coeff'] = df['mois'].map(mois_ustensiles_coefficients)

    # Coefficients régionaux
    df['region_coeff'] = df['profil_cotier'].map(region_ustensiles_coefficients)

    # Bonus pour les zones touristiques en haute saison
    if 'saison_toristique' in df.columns:
        df.loc[(df['saison_toristique'] == 'Haute') & ((df['mois'] >= 6) & (df['mois'] <= 8)), 'region_coeff'] *= 1.5

    # Coefficients pour jours spéciaux
    df['special_coeff'] = 1.0

    # Appliquer les coefficients de jour spécial individuellement
    for day_type in ['jour_ferie', 'Weekend', 'vacances_scolaires', 'ramadan', 'new_year']:
        if day_type in df.columns:
            mask = df[day_type] == 1
            if day_type in special_day_ustensiles_coefficients and mask.any():
                df.loc[mask, 'special_coeff'] = special_day_ustensiles_coefficients[day_type]

    # Pour les jours avec plusieurs types, prendre le coefficient le plus élevé et ajouter un bonus
    if 'jour_ferie' in df.columns and 'Weekend' in df.columns:
        both_mask = (df['jour_ferie'] == 1) & (df['Weekend'] == 1)
        if both_mask.any():
            df.loc[both_mask, 'special_coeff'] = max(
                special_day_ustensiles_coefficients['jour_ferie'],
                special_day_ustensiles_coefficients['Weekend']
            ) * 1.2  # Bonus de 20% pour combinaison (plus élevé que jardinage car impact plus fort)

    # Effet des séquences météo favorables (plusieurs jours consécutifs de beau temps)
    df['seq_effect'] = 1.0
    if all(col in df.columns for col in ['temp_jour_lag1', 'temp_jour_lag3', 'precip_jour_lag1', 'precip_jour_lag3']):
        sequence_favorable = (
            (df['temp_jour'] >= 25) &
            (df['temp_jour_lag1'] >= 25) &
            (df['temp_jour_lag3'] >= 25) &
            (df['precip_jour'] < 2) &
            (df['precip_jour_lag1'] < 2) &
            (df['precip_jour_lag3'] < 2)
        )
        df.loc[sequence_favorable, 'seq_effect'] = 1.3  # +30% après 4jours de beau temps

    # Effet de la densité urbaine
    densite_map = {
        'Grande métropole': 1.3,  # Plus de gens = plus d'événements sociaux
        'Ville moyenne': 1.1,     # Bon niveau d'activités sociales
        'Petite ville': 0.9     # Moins d'événements de grande ampleur
    }
    if 'densite_urbaine' in df.columns:
        df['densite_effect'] = df['densite_urbaine'].map(densite_map)
    else:
        df['densite_effect'] = 1.0

    # Calcul final des ventes ajustées
    df['ustensiles_jetables'] = (
        df['ustensiles_jetables'] *
        df['mois_coeff'] *
        df['region_coeff'] *
        df['special_coeff'] *
        df['seq_effect'] *
        df['densite_effect']
    )

    # Limitation des ventes journalières selon le type de jour
    # Jours normaux
    df.loc[
        (df['special_coeff'] == 1.0),
        'ustensiles_jetables'
    ] = df.loc[
        (df['special_coeff'] == 1.0),
        'ustensiles_jetables'
    ].clip(upper=max_ventes_jour_normal)

    # Jours spéciaux (Weekend, jours fériés)
    df.loc[
        (df['special_coeff'] > 1.0),
        'ustensiles_jetables'
    ] = df.loc[
        (df['special_coeff'] > 1.0),
        'ustensiles_jetables'
    ].clip(upper=max_ventes_jour_special)
    # Nettoyer les colonnes temporaires utilisées pour les calculs
    cols_to_drop = ['mois_coeff',
        'region_coeff',
        'special_coeff',
        'seq_effect',
        'densite_effect']
    return df.drop(columns=cols_to_drop, errors='ignore')


def ajouter_bruit_ustensiles(df, niveau_bruit=0.12):
    """
    Ajoute un bruit réaliste aux données de ventes d'ustensiles jetables.
    Le niveau de bruit est légèrement plus élevé que pour le jardinage car les achats
    d'ustensiles jetables sont plus impulsifs et variables.
    """
    df_avec_bruit = df.copy()

    # Générer des facteurs multiplicatifs avec une distribution normale
    facteurs_multiplicatifs = np.random.normal(1.0, niveau_bruit, size=len(df))

    # Appliquer le bruit multiplicatif
    df_avec_bruit["ustensiles_jetables"] = df["ustensiles_jetables"] * facteurs_multiplicatifs

    # S'assurer que toutes les valeurs sont positives
    df_avec_bruit["ustensiles_jetables"] = np.maximum(1, df_avec_bruit["ustensiles_jetables"])
    # Check for NaN or inf values
    print(df_avec_bruit[~np.isfinite(df_avec_bruit["ustensiles_jetables"])])
    # Arrondir aux entiers
    df_avec_bruit["ustensiles_jetables"] = df_avec_bruit["ustensiles_jetables"].round().astype(int)

    return df_avec_bruit


def print_statistiques_ustensiles(df):
    """Affiche les statistiques de vente des ustensiles jetables pour vérification."""
    print("=== STATISTIQUES DES VENTES D'USTENSILES JETABLES ===")
    print(f"Ventes moyennes globales: {df['ustensiles_jetables'].mean():.2f}")

    # Par mois si disponible
    if 'mois' in df.columns:
        for mois in sorted(df['mois'].unique()):
            print(f"Ventes moyennes au mois {mois}: {df.loc[df['mois'] == mois, 'ustensiles_jetables'].mean():.2f}")

    # Par type de jour
    if 'Weekend' in df.columns:
        print(f"Ventes moyennes pendant Weekend: {df.loc[df['Weekend'] == 1, 'ustensiles_jetables'].mean():.2f}")
        print(f"Ventes moyennes hors weekend: {df.loc[df['Weekend'] == 0, 'ustensiles_jetables'].mean():.2f}")

    if 'jour_ferie' in df.columns:
        print(f"Ventes moyennes pendant jours fériés: {df.loc[df['jour_ferie'] == 1, 'ustensiles_jetables'].mean():.2f}")
        print(f"Ventes moyennes hors jours fériés: {df.loc[df['jour_ferie'] == 0, 'ustensiles_jetables'].mean():.2f}")

    if 'vacances_scolaires' in df.columns:
        print(f"Ventes moyennes pendant vacances scolaires: {df.loc[df['vacances_scolaires'] == 1, 'ustensiles_jetables'].mean():.2f}")
        print(f"Ventes moyennes hors vacances scolaires: {df.loc[df['vacances_scolaires'] == 0, 'ustensiles_jetables'].mean():.2f}")

    # Par région si disponible
    if 'profil_cotier' in df.columns:
        for region in df['profil_cotier'].unique():
            print(f"Ventes moyennes en région {region}: {df.loc[df['profil_cotier'] == region, 'ustensiles_jetables'].mean():.2f}")

    print("================================================")


def generer_ventes_ustensiles_completes(df, base_ventes=50, niveau_bruit=0.12):
    """Processus complet de génération des ventes d'ustensiles jetables."""
    # 1. Génération des ventes de base avec impact climatique
    df = generer_ventes_ustensiles_base(df, base_ventes=base_ventes)

    # 2. Ajustement des ventes avec tous les facteurs
    df = ajuster_ventes_ustensiles(df)

    # 3. Ajout de bruit pour un modèle plus réaliste
    df = ajouter_bruit_ustensiles(df, niveau_bruit=niveau_bruit)

    # 4. Afficher les statistiques pour vérification
    print_statistiques_ustensiles(df)

    return df


df = generer_ventes_ustensiles_completes(df, base_ventes=50, niveau_bruit=0.12)

Empty DataFrame
Columns: [date, city, profil_cotier, densite_urbaine, region_climatique, temp_jour, humidity_jour, wind_speed_jour, precip_jour, point de rosée/jour, temp_jour_lag1, humidity_jour_lag1, wind_speed_jour_lag1, precip_jour_lag1, temp_jour_lag3, humidity_jour_lag3, wind_speed_jour_lag3, precip_jour_lag3, temp_jour_lag7, humidity_jour_lag7, wind_speed_jour_lag7, precip_jour_lag7, mois, Jour, Weekend, jour_ferie, saison, vacances_scolaires, ramadan, new_year, saison_toristique, Indice de Chaleur, refroidissement éolien, Indice d’inconfort, Indice de Vague de Chaleur, Indice de Vague de Froid , Indice de Pluie Intense, Indice de Tempête, Indice de sécheresse, Indice climatique thermique universel (ICTU), boissons_fraiches, snacks_sucré, snacks_salé, produits_laitiers_frais(yaourts, fromage blanc), produits_laitiers_fromage, produits_laitiers(lait,boissons lactées), produits_laitiers(lait glacé,crème glacée), boissons_chaudes, produits anti_moustiques, produits_jardinage, trend

In [ ]:
print(df['ustensiles_jetables'].unique())

[ 11   8  10  32   9  13  31  21  20   7  15  19  12  17  35  18  25   5
  22  16  30   6  14  34  45  41  24  26  48  23  33  51  42  28  38  37
  79  40  36  49  29  56  60  39  27  80  55  53  46  54  59  62  52  44
  43  50  61  57  47  63  97  76  78 100  74  58  87  65  98  66  82  71
  81  75  70  69  89  84  73  85  67  83  93  86 101  91  77  64  90  72
  68  88  92 106  94  96  95 103 102 110 104  99 108 107 105 113 109   4
   3   2   1 111 117 112 114]


# Modélisation des Ventes de Crème Solaire:Approche Climatique et Démographique

Ce module génère des données synthétiques réalistes pour les ventes de crème solaire en intégrant les facteurs météorologiques et géographiques.

---

## 1. Impact des Conditions Météorologiques

### Effet de la Température
- **<15°C** : Coefficient 0.3 (ventes très faibles)
- **15-25°C** : Augmentation progressive de 0.5 à 0.8
- **>25°C** : Forte augmentation (jusqu'à +50% avec coefficient max de 1.5)

```python
if temp_jour < 15:
    temp_effect = 0.3
elif 15 <= temp_jour < 25:
    temp_effect = 0.5 + 0.03 * (temp_jour - 15)
else:
    temp_effect = 0.8 + 0.05 * (temp_jour - 25)
```
## Effet de l'Humidité

- **Impact** :  
  Réduction de 0.2% par point d'humidité au-dessus de 60%  
  *Exemple* : À 70% d'humidité → -2% des ventes (10 points × 0.2%)

- **Plancher garanti** :  
  Minimum de 30% des ventes de base conservé quelles que soient les conditions

---

## 2. Facteurs Géographiques

### Densité Urbaine

| Type de Zone       | Coefficient | Explication                     |
|--------------------|-------------|---------------------------------|
| Grande métropole   | 1.3         | +30% (population dense)        |
| Ville moyenne      | 1.2         | +20%                           |
| Petite ville       | 1.0         | Valeur de référence            |

---

## 3. Contrôles et Validation

### Limites des Ventes
- **Seuil maximal** : 80 unités/jour  
  *Pour éviter des valeurs irréalistes*

### Modélisation du Réalisme
- **Bruit aléatoire** : ±10%  
  *Simule les variations naturelles du marché*
---

In [ ]:

max_ventes_jour_normal = 80
max_ventes_jour_special = 80

def meteo_creme_solaire_impact(temp_jour, humidity_jour):
    """
    Calcule l'impact météorologique sur les ventes de crème solaire.

    """
    # Impact de la température
    if temp_jour < 15:
        temp_effect = 0.3
    elif 15 <= temp_jour < 25:
        temp_effect = 0.5 + 0.03 * (temp_jour - 15)
    else:
        temp_effect = 0.8 + 0.05 * (temp_jour - 25)
        temp_effect = min(temp_effect, 1.5)  # Plafonné à +50%

    # Effet humidité (réduit légèrement les ventes)
    humidity_effect = 1 - 0.002 * max(0, humidity_jour - 60)

    return max(0.3, temp_effect * humidity_effect)

def calcul_impact_climatique(temp_jour, humidity_jour):
    """Calcul de l'impact climatique pour les ventes de crème solaire."""
    impact = meteo_creme_solaire_impact(temp_jour, humidity_jour)
    return impact

def generer_ventes_creme_solaire(df, base_ventes=30):
    """Génère les ventes de crème solaire avec impact climatique de base."""
    df['creme_solaire'] = df.apply(
        lambda row: calcul_impact_climatique(
            row['temp_jour'],
            row['humidity_jour']

        ) * base_ventes ,
        axis=1
    )
    return df

def ajuster_ventes(df):
    """Ajuste les ventes de crème solaire en fonction de divers facteurs."""


    # Effet de la densité urbaine
    densite_map = {
        'Grande métropole': 1.3,
        'Ville moyenne': 1.2,
        'Petite ville': 1.0
    }
    df['densite_effect'] = df['densite_urbaine'].map(densite_map)

    # Calcul final des ventes ajustées
    df['creme_solaire'] = df['creme_solaire']  * df['densite_effect']

    # Limitation des ventes journalières
    df.loc[df['creme_solaire'] > max_ventes_jour_normal, 'creme_solaire'] = max_ventes_jour_normal

    return df

def ajouter_bruit(df, niveau_bruit=0.1):
    """Ajoute un bruit réaliste aux données de ventes de crème solaire."""
    df_avec_bruit = df.copy()

    # Générer des facteurs multiplicatifs avec une distribution normale
    facteurs_multiplicatifs = np.random.normal(1.0, niveau_bruit, size=len(df))

    # Appliquer le bruit multiplicatif
    df_avec_bruit["creme_solaire"] = df["creme_solaire"] * facteurs_multiplicatifs

    # S'assurer que toutes les valeurs sont positives
    df_avec_bruit["creme_solaire"] = np.maximum(1, df_avec_bruit["creme_solaire"])

    # Arrondir aux entiers
    df_avec_bruit["creme_solaire"] = df_avec_bruit["creme_solaire"].round().astype(int)

    return df_avec_bruit

def processus_de_generation_complet(df, base_ventes=80, niveau_bruit=0.1):
    """Processus complet de génération des ventes de crème solaire."""

    # 1. Génération des ventes de base avec impact climatique
    df = generer_ventes_creme_solaire(df, base_ventes=base_ventes)

    # 2. Ajustement des ventes avec tous les facteurs
    df = ajuster_ventes(df)

    # 3. Ajout de bruit pour un modèle plus réaliste
    df = ajouter_bruit(df, niveau_bruit=niveau_bruit)

    # Afficher les statistiques pour vérification
    print_statistiques(df)

    return df

def print_statistiques(df):
    """Affiche les statistiques de vente de crème solaire pour vérification."""
    print("=== STATISTIQUES DES VENTES DE CRÈME SOLAIRE ===")
    print(f"Ventes moyennes globales: {df['creme_solaire'].mean():.2f}")

    # Par type de jour
    for day_type in ['Weekend', 'jour_ferie', 'vacances_scolaires']:
        if day_type in df.columns:
            print(f"Ventes moyennes pendant {day_type}: {df.loc[df[day_type] == 1, 'creme_solaire'].mean():.2f}")
            print(f"Ventes moyennes hors {day_type}: {df.loc[df[day_type] == 0, 'creme_solaire'].mean():.2f}")

    # Par région si disponible
    if 'profil_cotier' in df.columns:
        for region in df['profil_cotier'].unique():
            print(f"Ventes moyennes en région {region}: {df.loc[df['profil_cotier'] == region, 'creme_solaire'].mean():.2f}")

    print("===============================")

df = processus_de_generation_complet(df, base_ventes=50, niveau_bruit=0.1)

=== STATISTIQUES DES VENTES DE CRÈME SOLAIRE ===
Ventes moyennes globales: 38.26
Ventes moyennes pendant Weekend: 38.25
Ventes moyennes hors Weekend: 38.26
Ventes moyennes pendant jour_ferie: 43.93
Ventes moyennes hors jour_ferie: 37.98
Ventes moyennes pendant vacances_scolaires: 46.77
Ventes moyennes hors vacances_scolaires: 35.08
Ventes moyennes en région Hors côtier: 41.88
Ventes moyennes en région Intérieure: 35.56
Ventes moyennes en région Côtière: 40.91


In [ ]:
print(df['creme_solaire'].unique())

[ 21  13  20  16  17  18  14  15  23  35  26  37  25  33  27  32  28  31
  29  30  24  39  38  36  22  34  42  40  43  45  44  12  19  41  11  47
  49  46  55  53  48  50  51  52  56  54  58  59  65  62  63  57  64  67
  61  66  60  70  71  69  73  68  79  72  75  81  74  77  80  76  82  78
  83  86  90  87  85  89  84  96   9  92  91  94  88  10  95  93 100  97]


In [ ]:
df= df.to_csv('data1.csv',index=False)

In [ ]:
df=pd.read_csv('data1.csv')
print(df.shape)

(43769, 52)


# Modélisation des Ventes d'Équipement d'Urgence: Approche Basée sur les Conditions Extrêmes

Ce module génère des données réalistes pour les ventes de produits d'urgence (bougies, lampes de poche, piles) en fonction des conditions météorologiques critiques.

---

## 1. Impact des Conditions Météorologiques

### Effet de la Température
| Conditions Thermiques | Coefficient | Impact |
|-----------------------|-------------|--------|
| Températures extrêmes (<0°C ou >35°C) | 1.3 | +30% |
| Températures limites (<10°C ou >30°C) | 1.1 | +10% |
| Températures normales | 1.0 | Neutre |

### Effet des Précipitations
| Niveau de Pluie | Coefficient | Impact |
|-----------------|-------------|--------|
| >60mm | 1.4 | +40% |
| 30-60mm | 1.2 | +20% |
| <30mm | 1.0 | Neutre |

### Effet du Vent
| Vitesse du Vent | Coefficient | Impact |
|-----------------|-------------|--------|
| >30 km/h | 1.3 | +30% |
| 15-30 km/h | 1.15 | +15% |
| <15 km/h | 1.0 | Neutre |

**Plafond global** : +100% (coefficient max de 2.0)

---

## 2. Facteurs Démographiques

### Densité Urbaine
| Type de Zone | Coefficient | Explication |
|--------------|-------------|-------------|
| Grande métropole | 1.3 | +30% (population dense) |
| Ville moyenne | 1.2 | +20% |
| Petite ville | 1.0 | Référence |

---

## 3. Paramètres de Contrôle

- **Ventes de base** : 20 unités/jour
- **Maximum journalier** : 100 unités
- **Bruit aléatoire** : ±10% pour le réalisme

---

In [ ]:
'''Équipement d'urgence(Bougies, lampes de poche ,Piles, batteries rechargeables...)'''

def meteo_equipement_impact(temp_jour, precip_jour, wind_speed_jour):
    """
    Calcule l'impact météorologique sur les ventes d'équipement d'urgence.

    Les conditions météorologiques extrêmes augmentent la vente d'équipement.
    """
    # Impact de la température
    if temp_jour < 0 or temp_jour > 35:
        temp_effect = 1.3  # +30% en cas de températures extrêmes
    elif temp_jour < 10 or temp_jour > 30:
        temp_effect = 1.1  # +10% en cas de températures limites
    else:
        temp_effect = 1.0  # Impact neutre

    # Impact des précipitations
    if precip_jour > 60:  # Précipitations importantes
        precip_effect = 1.4  # +40%
    elif precip_jour > 30:
        precip_effect = 1.2  # +20%
    else:
        precip_effect = 1.0  # Neutre

    # Impact du vent
    if wind_speed_jour > 30:  # Vent fort
        wind_effect = 1.3  # +30%
    elif wind_speed_jour > 15:
        wind_effect = 1.15  # +15%
    else:
        wind_effect = 1.0  # Neutre

    return min(2.0, temp_effect * precip_effect * wind_effect)  # Plafonné à +100%

def calcul_impact_climatique(temp_jour, precip_jour, wind_speed_jour):
    """Calcul de l'impact climatique spécifique à l'équipement d'urgence"""
    return meteo_equipement_impact(temp_jour, precip_jour, wind_speed_jour)

def generer_ventes_equipement_urgence(df, base_ventes=20):
    """Génère les ventes avec impact climatique de base"""
    df['equipement_urgence'] = df.apply(
        lambda row: calcul_impact_climatique(
            row['temp_jour'],
            row['precip_jour'],
            row['wind_speed_jour']
        ) * base_ventes,
        axis=1
    )
    return df

def ajuster_ventes(df):
    """Ajuste les ventes en fonction de divers facteurs"""

    # Effet de la densité de population
    densite_map = {
        'Grande métropole': 1.3,
        'Ville moyenne': 1.2,
        'Petite ville': 1.0,
    }
    df['densite_effect'] = df['densite_urbaine'].map(densite_map)

    # Calcul final des ventes ajustées
    df['equipement_urgence'] = (
        df['equipement_urgence'] *

        df['densite_effect']
    )

    # Limitation des ventes journalières
    max_ventes_jour = 100
    df['equipement_urgence'] = df['equipement_urgence'].clip(upper=max_ventes_jour)

    return df

def ajouter_bruit(df, niveau_bruit=0.1):
    """Ajoute un bruit réaliste aux données"""
    df_avec_bruit = df.copy()

    # Générer des facteurs multiplicatifs avec distribution normale
    facteurs_multiplicatifs = np.random.normal(1.0, niveau_bruit, size=len(df))

    # Appliquer le bruit multiplicatif
    df_avec_bruit["equipement_urgence"] = df["equipement_urgence"] * facteurs_multiplicatifs

    # S'assurer que toutes les valeurs sont positives
    df_avec_bruit["equipement_urgence"] = np.maximum(1, df_avec_bruit["equipement_urgence"])

    # Arrondir aux entiers
    df_avec_bruit["equipement_urgence"] = df_avec_bruit["equipement_urgence"].round().astype(int)

    return df_avec_bruit

def processus_de_generation_complet(df, base_ventes=20, niveau_bruit=0.1):
    """Processus complet de génération de données"""
    # 1. Génération des ventes de base avec impact climatique
    df = generer_ventes_equipement_urgence(df, base_ventes=base_ventes)

    # 2. Ajustement des ventes avec tous les facteurs
    df = ajuster_ventes(df)

    # 3. Ajout de bruit pour un modèle plus réaliste
    df = ajouter_bruit(df, niveau_bruit=niveau_bruit)



    return df




df = processus_de_generation_complet(df, base_ventes=20, niveau_bruit=0.1)
print(df['equipement_urgence'].unique())

[25 18 24 26 22 19 20 21 23 28 17 16 29 27 31 15 30 33 34 32 35 36 38 37
 13 14 43 39 41 12 42 49 44 46 40 45 50 54 51 57 47 52 48 55]


In [ ]:
df= df.to_csv('data1.csv',index=False)

In [ ]:
df= pd.read_csv('data.csv')

# Modélisation des Ventes de Poudre Anti-Transpirante :Approche Climatique et Démographique

Ce module génère des données synthétiques réalistes pour les ventes de poudre anti-transpirante en fonction des conditions météorologiques et des caractéristiques géographiques.

---

## 1. Impact des Conditions Météorologiques

### Effet de la Température
| Plage de Température | Coefficient | Impact |
|----------------------|-------------|--------|
| <15°C | 0.6 | -40% |
| 15-25°C | 0.8 à 1.0 | +2% par °C |
| 25-35°C | 1.0 à 1.4 | +4% par °C |
| >35°C | 1.4 | +40% (plafond) |

### Autres Facteurs Climatiques
- **Précipitations** : Réduction jusqu'à -30% (minimum 70% des ventes)
- **Vent chaud** (>30°C et vent >25km/h) : +10%
- **Humidité** :
  - Temps chaud (>25°C) : +1.5% par point >60%
  - Temps frais : -0.5% par point >60%

**Minimum garanti** : 40% des ventes de base

---

## 2. Facteurs Géographiques

### Densité Urbaine
| Type de Zone | Coefficient | Impact |
|--------------|-------------|--------|
| Grande métropole | 1.3 | +30% |
| Ville moyenne | 1.1 | +10% |
| Petite ville | 1.0 | Référence |

### Région Climatique
| Type de Climat | Coefficient | Impact |
|----------------|-------------|--------|
| Aride/désertique | 1.6 | +60% |
| Semi-aride | 1.4 | +40% |
| Méditerranéen | 1.2 | +20% |

---

## 3. Paramètres de Contrôle

- **Ventes de base** : 25 unités/jour
- **Maximum journalier** : 70 unités
- **Bruit aléatoire** : ±7% pour le réalisme
- **Effet de tendance** : +20% après une hausse brutale de température (>7°C en un jour)

---

In [ ]:
def meteo_poudre_impact(temp_jour, humidity_jour, precip_jour, wind_speed_jour):

    # Effet température
    if temp_jour < 15:
        temp_effect = 0.6  # Réduction des ventes par temps froid
    elif 15 <= temp_jour < 25:
        temp_effect = 0.8 + 0.02 * (temp_jour - 15)  # Augmentation progressive
    elif 25 <= temp_jour < 35:
        temp_effect = 1.0 + 0.04 * (temp_jour - 25)  # Augmentation plus rapide
    else:
        temp_effect = 1.4  # Plafonné à +40% au-delà de 35°C

    # Effet précipitations
    precip_effect = max(0.7, 1 - (precip_jour * 0.05))  # Minimum 70% des ventes normales

    # Effet vent
    wind_effect = 1.0
    if temp_jour > 30 and wind_speed_jour > 25:
        wind_effect = 1.1  # Léger boost par vent chaud

    # Effet humidité
    if temp_jour >= 25:
        # Impact plus significatif de l'humidité par temps chaud
        humidity_effect = 1 + 0.015 * max(0, humidity_jour - 60)
    else:
        # Impact modéré par temps plus frais
        humidity_effect = 1 - 0.005 * max(0, humidity_jour - 60)

    # Combinaison des effets
    return max(0.4, temp_effect * precip_effect * wind_effect * humidity_effect)  # Minimum 40% des ventes de base

def calcul_impact_climatique(temp_jour, humidity_jour, precip_jour, wind_speed_jour):

    impact = meteo_poudre_impact(temp_jour, humidity_jour, precip_jour, wind_speed_jour)
    return impact

def generer_ventes_poudre(df, base_ventes=30):
    """Génère les ventes avec impact climatique de base."""

    df['poudre_antitranspirante'] = df.apply(
        lambda row: calcul_impact_climatique(
            row['temp_jour'],
            row['humidity_jour'],
            row['precip_jour'],
            row['wind_speed_jour']
        ) * base_ventes,
        axis=1
    )
    return df

def ajuster_ventes(df):
    """Ajuste les ventes de base en fonction des autres features."""



    # Effet de la densité urbaine
    densite_map = {
        'Grande métropole': 1.3,
        'Ville moyenne': 1.1,
        'Petite ville': 1.0
    }
    df['densite_effect'] = df['densite_urbaine'].map(densite_map)

    # Effet de la région climatique
    climat_map = {
        'Méditerranéenne': 1.2,
        'Semi-aride': 1.4,
        'Aride/désertique': 1.6
    }
    df['climat_effect'] = df['region_climatique'].map(climat_map)

    # Tendance des ventes
    df['trend_effect'] = 1.0
    if 'temp_jour_lag1' in df.columns:
        delta_temp = df['temp_jour'] - df['temp_jour_lag1']
        df.loc[(delta_temp > 7) & (df['temp_jour'] > 30), 'trend_effect'] = 1.2

    # Calcul final des ventes ajustées
    df['poudre_antitranspirante'] = df['poudre_antitranspirante']  * df['trend_effect'] * df['densite_effect'] * df['climat_effect']

    # Limitation des ventes journalières
    max_ventes_jour = 70  # Valeur à ajuster selon vos besoins
    df['poudre_antitranspirante'] = df['poudre_antitranspirante'].clip(upper=max_ventes_jour)

    return df

def ajouter_bruit(df, niveau_bruit=0.07):
    """Ajoute un bruit réaliste aux données."""
    import numpy as np

    df_avec_bruit = df.copy()

    # Générer des facteurs multiplicatifs avec une distribution normale
    facteurs_multiplicatifs = np.random.normal(1.0, niveau_bruit, size=len(df))

    # Appliquer le bruit multiplicatif
    df_avec_bruit["poudre_antitranspirante"] = df["poudre_antitranspirante"] * facteurs_multiplicatifs

    # S'assurer que toutes les valeurs sont positives
    df_avec_bruit["poudre_antitranspirante"] = np.maximum(1, df_avec_bruit["poudre_antitranspirante"])

    # Arrondir aux entiers
    df_avec_bruit["poudre_antitranspirante"] = df_avec_bruit["poudre_antitranspirante"].round().astype(int)

    return df_avec_bruit

def print_statistiques(df):
    """Affiche les statistiques de vente pour vérification."""
    print("=== STATISTIQUES DES VENTES DE POUDRE ANTI-TRANSPIRANTE ===")
    print(f"Ventes moyennes globales: {df['poudre_antitranspirante'].mean():.2f}")

    # Par région climatique
    if 'region_climatique' in df.columns:
        for climat in df['region_climatique'].unique():
            print(f"Ventes moyennes en zone {climat}: {df.loc[df['region_climatique'] == climat, 'poudre_antitranspirante'].mean():.2f}")

    print("===============================")

def processus_de_generation_complet(df, base_ventes=30, niveau_bruit=0.07):
    """Processus complet de génération de ventes."""

    # 1. Génération des ventes de base avec impact climatique
    df = generer_ventes_poudre(df, base_ventes=base_ventes)

    # 2. Ajustement des ventes avec tous les facteurs
    df = ajuster_ventes(df)

    # 3. Ajout de bruit pour un modèle plus réaliste
    df = ajouter_bruit(df, niveau_bruit=niveau_bruit)

    # Afficher les statistiques pour vérification
    print_statistiques(df)

    return df


df = processus_de_generation_complet(df, base_ventes=25, niveau_bruit=0.07)

=== STATISTIQUES DES VENTES DE POUDRE ANTI-TRANSPIRANTE ===
Ventes moyennes globales: 30.77
Ventes moyennes en zone Méditerranéenne: 30.29
Ventes moyennes en zone Semi-aride: 29.89
Ventes moyennes en zone Aride/désertique: 34.11


In [ ]:
print(df['poudre_antitranspirante'].unique())

[23 25 19 20 21 24 22 32 26 31 30 28 33 27 36 34 35 39 29 41 38 37 15 18
 17 16 40 42 49 44 45 43 46 47 48 55 50 51 52 56 54 53 59 57 58 60 61 14
 13 12 11 62 65 64 68 63 70 67 10]


In [ ]:
df=df.to_csv('data1.csv',index=False)

In [ ]:
import pandas as pd
df=pd.read_csv('data.csv')

# Modélisation des Ventes de Produits de Cuisson au four : Approche Saisonnière et Culturelle

Ce module génère des données synthétiques réalistes pour les ventes de produits de cuisson au four en intégrant les facteurs météorologiques, saisonniers et culturels.

---

## 1. Impact des Conditions Météorologiques

### Effet de la Température
| Plage de Température | Coefficient | Impact |
|----------------------|-------------|--------|
| <10°C | 0.7 | +30% (envie de pâtisseries chaudes) |
| 10-20°C | 0.8 à 1.0 | +2% par °C |
| 20-30°C | 1.0 à 1.3 | +3% par °C (zone optimale) |
| >30°C | 0.9 | Légère baisse |

### Autres Facteurs Climatiques
- **Précipitations** : +5% par mm (max +30%)
- **Vent fort** (>20 km/h) : -10%
- **Humidité** :
  - Temps chaud (>20°C) : -0.5% par point >60%
  - Temps frais : +0.3% par point >50%

**Minimum garanti** : 40% des ventes de base

---

## 2. Facteurs Saisonniers et Culturels

### Saisonnalité Mensuelle
| Période | Coefficient | Explication |
|---------|-------------|-------------|
| Hiver (déc-fév) | 1.4-1.8 | Pic des fêtes et temps froid |
| Été (juin-août) | 0.8-0.9 | Baisse saisonnière |
| Automne | 1.2-1.3 | Retour des envies de cuisson |

### Jours Spéciaux
| Événement | Coefficient | Bonus |
|-----------|-------------|-------|
| Nouvel An | 1.6 | +60% |
| Ramadan | 1.4 | +40% |
| Weekends | 1.2 | +20% |
| Combinaison férié+weekend | 1.32 | +10% supplémentaire |

---

## 3. Facteurs Démographiques

### Densité Urbaine
| Type de Zone | Coefficient | Impact |
|--------------|-------------|--------|
| Grande métropole | 1.15 | +15% |
| Ville moyenne | 1.1 | +10% |
| Petite ville | 1.0 | Référence |

---

## 4. Paramètres de Contrôle

- **Ventes de base** : 50 unités/jour
- **Maximum journalier** : 100 unités
- **Bruit aléatoire** : ±8% pour le réalisme
- **Effet de tendance** : +20% après une hausse brutale de température (>5°C en un jour)

---

In [ ]:
import numpy as np

# Coefficients pour différents jours spéciaux
special_day_coefficients = {
    'jour_ferie': 1.2,
    'Weekend': 1.2,
    'vacances_scolaires': 1.1,
    'ramadan': 1.4,
    'new_year': 1.6
}
 # Coefficients mensuels (pic en hiver et pendant les fêtes)
monthly_coeff = {
        1: 1.6, 2: 1.4, 3: 1.2,   # Hiver
        4: 1.0, 5: 0.9, 6: 0.8,    # Printemps
        7: 0.8, 8: 0.9, 9: 1.0,    # Été
        10: 1.2, 11: 1.3, 12: 1.8  # Automne/Hiver
    }


# Constante globale pour limiter les ventes maximales
max_ventes_jour_normal = 100
max_ventes_jour_special = 100

def meteo_patisserie_impact(temp_jour, humidity_jour, precip_jour, wind_speed_jour):
    """
    Calcule l'impact des conditions météorologiques sur les ventes de produits de cuisson.
    """
    # Effet température de base
    if temp_jour < 10:
        temp_effect = 0.7  # Hausse des envies de pâtisseries chaudes
    elif 10 <= temp_jour < 20:
        temp_effect = 0.8 + 0.02 * (temp_jour - 10)  # Augmentation graduelle
    elif 20 <= temp_jour < 30:
        temp_effect = 1.0 + 0.03 * (temp_jour - 20)  # Température optimale
    else:
        temp_effect = 0.9  # Légère baisse par forte chaleur

    # Effet précipitations (favorise la cuisson intérieure)
    precip_effect = 1 + (precip_jour * 0.05)  # Augmentation avec les précipitations
    precip_effect = min(precip_effect, 1.3)  # Plafonné à +30%

    # Effet vent
    wind_effect = 1.0
    if wind_speed_jour > 20:
        wind_effect = 0.9  # Léger impact du vent fort

    # Effet humidité
    if temp_jour >= 20:
        humidity_effect = 1 - 0.005 * max(0, humidity_jour - 60)
    else:
        humidity_effect = 1 + 0.003 * max(0, humidity_jour - 50)

    # Combinaison des effets
    return max(0.4, temp_effect * precip_effect * wind_effect * humidity_effect)

def calcul_impact_climatique(temp_jour, humidity_jour, precip_jour, wind_speed_jour):
    """Calcul de l'impact climatique spécifique aux produits de cuisson."""
    impact = meteo_patisserie_impact(temp_jour, humidity_jour, precip_jour, wind_speed_jour)
    return impact

def generer_ventes_patisserie(df, base_ventes=50):
    """Génère les ventes avec impact climatique de base."""
    df['produits_cuisson'] = df.apply(
        lambda row: calcul_impact_climatique(
            row['temp_jour'],
            row['humidity_jour'],
            row['precip_jour'],
            row['wind_speed_jour']
        ) * base_ventes,
        axis=1
    )
    return df

def ajuster_ventes(df):
    """Ajuste les ventes de base en fonction de divers facteurs."""

    # Coefficients pour jours spéciaux
    df['special_coeff'] = 1.0


    # Appliquer les coefficients de jour spécial
    for day_type in ['jour_ferie', 'Weekend', 'vacances_scolaires', 'ramadan', 'new_year']:
        mask = df[day_type] == 1
        if day_type in special_day_coefficients and mask.any():
            df.loc[mask, 'special_coeff'] = special_day_coefficients[day_type]

    # Gestion des jours combinés
    if 'jour_ferie' in df.columns and 'Weekend' in df.columns:
        both_mask = (df['jour_ferie'] == 1) & (df['Weekend'] == 1)
        if both_mask.any():
            df.loc[both_mask, 'special_coeff'] = max(
                special_day_coefficients['jour_ferie'],
                special_day_coefficients['Weekend']
            ) * 1.1  # Bonus de 10% pour combinaison
    # Coefficient mensuel

    df['mois_coeff'] = df['mois'].map(monthly_coeff)
    # Effet de tendance
    df['trend_effect'] = 1.0
    if 'temp_jour_lag1' in df.columns:
        delta_temp = df['temp_jour'] - df['temp_jour_lag1']
        df.loc[(delta_temp > 5) & (df['temp_jour'] > 25), 'trend_effect'] = 1.2

    # Effet de la densité urbaine
    densite_map = {
        'Grande métropole': 1.15,
        'Ville moyenne': 1.1,
        'Petite ville': 1.0
    }
    df['densite_effect'] = df['densite_urbaine'].map(densite_map)



    # Augmentations spéciales pour certains événements
    df.loc[df['new_year'] == 1, 'special_coeff'] = special_day_coefficients['new_year'] * 1.3
    df.loc[df['ramadan'] == 1, 'special_coeff'] = special_day_coefficients['ramadan'] * 1.2

    # Calcul final des ventes ajustées
    df['produits_cuisson'] = df['produits_cuisson'] * df['special_coeff'] * df['trend_effect'] * df['densite_effect'] * df['mois_coeff']

    # Limitation des ventes journalières
    df.loc[
        (~df['new_year'].astype(bool)) & (~df['ramadan'].astype(bool)) & (df['produits_cuisson'] > max_ventes_jour_normal),
        'produits_cuisson'
    ] = max_ventes_jour_normal

    df.loc[
        (df['new_year'].astype(bool) | df['ramadan'].astype(bool)),
        'produits_cuisson'
    ] = df.loc[
        (df['new_year'].astype(bool) | df['ramadan'].astype(bool)),
        'produits_cuisson'
    ].clip(upper=max_ventes_jour_special)

    return df

def ajouter_bruit(df, niveau_bruit=0.08):
    """Ajoute un bruit réaliste aux données."""
    df_avec_bruit = df.copy()

    facteurs_multiplicatifs = np.random.normal(1.0, niveau_bruit, size=len(df))

    df_avec_bruit["produits_cuisson"] = df["produits_cuisson"] * facteurs_multiplicatifs

    df_avec_bruit["produits_cuisson"] = np.maximum(1, df_avec_bruit["produits_cuisson"])
    df_avec_bruit["produits_cuisson"] = df_avec_bruit["produits_cuisson"].round().astype(int)

    return df_avec_bruit

def processus_de_generation_complet(df, base_ventes=50, niveau_bruit=0.08):
    """
    Processus complet de génération de données de ventes.


    """
    # 1. Génération des ventes de base avec impact climatique
    df = generer_ventes_patisserie(df, base_ventes=base_ventes)

    # 2. Ajustement des ventes avec tous les facteurs
    df = ajuster_ventes(df)

    # 3. Ajout de bruit pour un modèle plus réaliste
    df = ajouter_bruit(df, niveau_bruit=niveau_bruit)

    # 4. Afficher les statistiques pour vérification
    print_statistiques(df)

    return df

def print_statistiques(df):
    """Affiche les statistiques de vente pour vérification."""
    print("=== STATISTIQUES DES VENTES DE PRODUITS DE CUISSON ===")
    print(f"Ventes moyennes globales: {df['produits_cuisson'].mean():.2f}")

    # Par type de jour
    if 'new_year' in df.columns:
        print(f"Ventes moyennes pendant Nv année: {df.loc[df['new_year'] == 1, 'produits_cuisson'].mean():.2f}")
        print(f"Ventes moyennes hors Nv année: {df.loc[df['new_year'] == 0, 'produits_cuisson'].mean():.2f}")

    if 'Weekend' in df.columns:
        print(f"Ventes moyennes pendant Weekend: {df.loc[df['Weekend'] == 1, 'produits_cuisson'].mean():.2f}")
        print(f"Ventes moyennes hors weekend: {df.loc[df['Weekend'] == 0, 'produits_cuisson'].mean():.2f}")
    if 'ramadan' in df.columns:
        print(f"Ventes moyennes pendant ramadan: {df.loc[df['ramadan'] == 1, 'produits_cuisson'].mean():.2f}")
        print(f"Ventes moyennes hors ramadan: {df.loc[df['ramadan'] == 0, 'produits_cuisson'].mean():.2f}")
    # Par région si disponible
    if 'mois' in df.columns:
        for mois in df['mois'].unique():
            print(f"Ventes moyennes en mois {mois}: {df.loc[df['mois'] == mois, 'produits_cuisson'].mean():.2f}")

    print("===============================")


df = processus_de_generation_complet(df, base_ventes=50, niveau_bruit=0.08)

=== STATISTIQUES DES VENTES DE PRODUITS DE CUISSON ===
Ventes moyennes globales: 72.20
Ventes moyennes pendant Nv année: 99.09
Ventes moyennes hors Nv année: 72.12
Ventes moyennes pendant Weekend: 77.66
Ventes moyennes hors weekend: 70.01
Ventes moyennes pendant ramadan: 93.04
Ventes moyennes hors ramadan: 70.31
Ventes moyennes en mois 2: 78.94
Ventes moyennes en mois 3: 75.17
Ventes moyennes en mois 4: 78.91
Ventes moyennes en mois 5: 66.93
Ventes moyennes en mois 6: 52.74
Ventes moyennes en mois 7: 47.80
Ventes moyennes en mois 8: 58.00
Ventes moyennes en mois 9: 68.38
Ventes moyennes en mois 10: 78.61
Ventes moyennes en mois 11: 79.87
Ventes moyennes en mois 12: 96.14
Ventes moyennes en mois 1: 85.10


In [ ]:
print(df['produits_cuisson'].unique())

[ 72  63  79  77  70  59  69  65  75  64  88  84  76  81  66  73  49  60
  57  80  74  67  92  96  58  62  85  68  86  51  78  45  95  91  97  71
  87 103  89  90  83  55  56  54  53  61  47  52  50  48  46  82  43  41
  40  42  44  37  38  36  39  94  98 104 100 108 112  93 101 111 106 117
  99 110 115 102 107 105 109  35  34  32  33  31  29 120 113 119 116 118
 125 121 114 122 124 123  28 130  30  27 127]


# Modélisation des Ventes de Produits d'Hygiène : Approche Climatique et Saisonnière

Ce module génère des données synthétiques réalistes pour les ventes de produits d'hygiène personnelle (déodorants et lingettes) en fonction des conditions météorologiques et des saisons.

---

## 1. Impact des Conditions Météorologiques

### Effet sur les Déodorants
| Conditions | Coefficient | Impact |
|------------|-------------|--------|
| Température >30°C | 1.4 | +40% |
| Température 25-30°C | 1.2 | +20% |
| Humidité >80% | 1.3 | +30% |
| Humidité 60-80% | 1.2 | +20% |

### Effet sur les Lingettes
| Conditions | Coefficient | Impact |
|------------|-------------|--------|
| Température >30°C | 1.3 | +30% |
| Précipitations >20mm | 1.2 | +20% |

**Plafond global** : +100% (coefficient max de 2.0)

---

## 2. Facteurs Saisonniers

### Coefficient Saison
| Saison | Coefficient | Impact |
|--------|-------------|--------|
| Été | 1.4 | +40% |
| Printemps/Automne | 1.2 | +20% |
| Hiver | 1.0 | Référence |

---

## 3. Paramètres de Contrôle

- **Ventes de base** :
  - Déodorants: 30 unités/jour
  - Lingettes: 25 unités/jour
- **Maximum journalier** :
  - Déodorants: 150 unités
  - Lingettes: 100 unités
- **Bruit aléatoire** : ±10% pour le réalisme

---

In [ ]:
def meteo_hygiene_impact(temp_jour, humidity_jour, precip_jour):
    """
    Calcule l'impact météorologique sur les ventes de produits d'hygiène et soins personnels.

    """
    # Impact de la température
    if temp_jour > 30:  # Chaleur importante
        temp_effect_deodorant = 1.4  # +40% de ventes de déodorants
        temp_effect_lingettes = 1.3  # +30% de lingettes rafraîchissantes
    elif temp_jour > 25:
        temp_effect_deodorant = 1.2  # +20%
        temp_effect_lingettes = 1.2  # +20%
    else:
        temp_effect_deodorant = 1.0
        temp_effect_lingettes = 1.0

    # Impact de l'humidité
    if humidity_jour > 80:  # Humidité très élevée
        humidity_effect = 1.3  # Augmentation des ventes de produits anti-transpiration
    elif humidity_jour > 60:
        humidity_effect = 1.2
    else:
        humidity_effect = 1.0

    # Impact des précipitations
    if precip_jour > 20:  # Précipitations significatives
        precip_effect_lingettes = 1.2  # +20% de lingettes de nettoyage
    else:
        precip_effect_lingettes = 1.0

    # Combinaison des effets
    return {
        'deodorant': min(2.0, temp_effect_deodorant * humidity_effect),
        'lingettes': min(2.0, temp_effect_lingettes * precip_effect_lingettes)
    }

def calcul_impact_climatique(temp_jour, humidity_jour, precip_jour):
    """Calcul de l'impact climatique spécifique aux produits d'hygiène"""
    return meteo_hygiene_impact(temp_jour, humidity_jour, precip_jour)

def generer_ventes_hygiene(df, base_ventes_deodorant=30, base_ventes_lingettes=25):
    """Génère les ventes avec impact climatique de base"""
    # Calculer l'impact climatique
    impact_climatique = df.apply(
        lambda row: calcul_impact_climatique(
            row['temp_jour'],
            row['humidity_jour'],
            row['precip_jour']
        ),
        axis=1
    )

    # Générer les ventes
    df['ventes_deodorant'] = impact_climatique.apply(lambda x: x['deodorant'] * base_ventes_deodorant)
    df['ventes_lingettes'] = impact_climatique.apply(lambda x: x['lingettes'] * base_ventes_lingettes)

    return df

def ajuster_ventes(df):
    """Ajuste les ventes en fonction de divers facteurs"""
    # Coefficient saison
    saison_map = {
        'été': 1.4,  # Plus de ventes en été
        'printemps/automne': 1.2,
        'hiver': 1.0
    }
    df['saison_effect'] = df['saison'].map(saison_map)

    # Calcul final des ventes ajustées
    df['ventes_deodorant'] = (
        df['ventes_deodorant'] *
        df['saison_effect']
    )

    df['ventes_lingettes'] = (
        df['ventes_lingettes'] *
        df['saison_effect']
    )

    # Limitation des ventes journalières
    max_ventes_deodorant = 150
    max_ventes_lingettes = 100

    df['ventes_deodorant'] = df['ventes_deodorant'].clip(upper=max_ventes_deodorant)
    df['ventes_lingettes'] = df['ventes_lingettes'].clip(upper=max_ventes_lingettes)

    return df

def ajouter_bruit(df, niveau_bruit=0.1):
    """Ajoute un bruit réaliste aux données"""
    df_avec_bruit = df.copy()

    # Générer des facteurs multiplicatifs avec distribution normale
    bruit_deodorant = np.random.normal(1.0, niveau_bruit, size=len(df))
    bruit_lingettes = np.random.normal(1.0, niveau_bruit, size=len(df))

    # Appliquer le bruit multiplicatif
    df_avec_bruit["ventes_deodorant"] = df["ventes_deodorant"] * bruit_deodorant
    df_avec_bruit["ventes_lingettes"] = df["ventes_lingettes"] * bruit_lingettes

    # S'assurer que toutes les valeurs sont positives
    df_avec_bruit["ventes_deodorant"] = np.maximum(1, df_avec_bruit["ventes_deodorant"])
    df_avec_bruit["ventes_lingettes"] = np.maximum(1, df_avec_bruit["ventes_lingettes"])

    # Arrondir aux entiers
    df_avec_bruit["ventes_deodorant"] = df_avec_bruit["ventes_deodorant"].round().astype(int)
    df_avec_bruit["ventes_lingettes"] = df_avec_bruit["ventes_lingettes"].round().astype(int)

    return df_avec_bruit

def processus_de_generation_complet(df, base_ventes_deodorant=30, base_ventes_lingettes=25, niveau_bruit=0.1):
    """Processus complet de génération de données"""
    # 1. Génération des ventes de base avec impact climatique
    df = generer_ventes_hygiene(df, base_ventes_deodorant, base_ventes_lingettes)

    # 2. Ajustement des ventes avec tous les facteurs
    df = ajuster_ventes(df)

    # 3. Ajout de bruit pour un modèle plus réaliste
    df = ajouter_bruit(df, niveau_bruit=niveau_bruit)

    # 4. Afficher les statistiques
    print_statistiques(df)

    return df

def print_statistiques(df):
    """Affiche les statistiques de vente pour vérification"""
    print("=== STATISTIQUES DES VENTES DE PRODUITS D'HYGIÈNE ===")
    print(f"Ventes moyennes de déodorants: {df['ventes_deodorant'].mean():.2f}")
    print(f"Ventes moyennes de lingettes: {df['ventes_lingettes'].mean():.2f}")


    # Par saison
    if 'saison' in df.columns:
        for saison in df['saison'].unique():
            print(f"Ventes moyennes de déodorants pendant {saison}: {df.loc[df['saison'] == saison, 'ventes_deodorant'].mean():.2f}")
            print(f"Ventes moyennes de lingettes pendant {saison}: {df.loc[df['saison'] == saison, 'ventes_lingettes'].mean():.2f}")

    print("===============================")


df = processus_de_generation_complet(df, base_ventes_deodorant=30, base_ventes_lingettes=25, niveau_bruit=0.1)

=== STATISTIQUES DES VENTES DE PRODUITS D'HYGIÈNE ===
Ventes moyennes de déodorants: 42.13
Ventes moyennes de lingettes: 32.99
Ventes moyennes de déodorants pendant hiver: 34.33
Ventes moyennes de lingettes pendant hiver: 25.77
Ventes moyennes de déodorants pendant printemps/automne: 40.60
Ventes moyennes de lingettes pendant printemps/automne: 31.64
Ventes moyennes de déodorants pendant été: 52.86
Ventes moyennes de lingettes pendant été: 42.78


In [ ]:
print('ventes_deodorant')
print(df['ventes_deodorant'].unique())
print('ventes_lingettes')
print(df['ventes_lingettes'].unique())

ventes_deodorant
[33 27 35 29 28 34 32 26 23 31 25 24 30 37 36 22 38 43 39 42 40 46 44 41
 49 47 48 45 51 53 52 60 50 56 54 58 55 57 59 65 61 62 67 63 66 64 68 70
 69 72 78 77 71 73 21 19 76 74 81 75 20]
ventes_lingettes
[24 29 22 28 25 21 23 27 26 30 33 20 19 31 32 34 35 38 36 37 44 39 40 41
 42 43 50 46 45 49 48 47 54 53 51 55 58 56 52 57 18 16 17 59 60 62 64 15
 61 72 68 65 14]


# Modélisation des Ventes de Produits d'Hydratation : Approche Climatique et Environnementale

Ce module génère des données synthétiques réalistes pour les ventes de produits d'hydratation en fonction des conditions météorologiques.

---

## 1. Impact des Conditions Météorologiques

### Effet de la Température
| Conditions Thermiques | Coefficient | Impact |
|-----------------------|-------------|--------|
| Température <10°C | 1.4 | +40% (peau sèche par froid) |
| Température >30°C | 1.2 | +20% (déshydratation) |
| Température 10-30°C | 1.0 | Neutre |

### Effet de l'Humidité
| Niveau d'Humidité | Coefficient | Impact |
|-------------------|-------------|--------|
| <40% | 1.3 | +30% (air sec) |
| >80% | 0.8 | -20% (air humide) |
| 40-80% | 1.0 | Neutre |

### Effet du Vent
| Vitesse du Vent | Coefficient | Impact |
|-----------------|-------------|--------|
| >20 km/h | 1.3 | +30% (effet desséchant) |
| ≤20 km/h | 1.0 | Neutre |

**Plafond global** : +100% (coefficient max de 2.0)

---

## 2. Paramètres de Contrôle

- **Ventes de base** : 30 unités/jour
- **Maximum journalier** : 100 unités
- **Bruit aléatoire** : ±10% pour le réalisme

---

In [ ]:
def meteo_hydratation_impact(temp_jour, humidity_jour, wind_speed_jour):
    """
    Calcule l'impact météorologique global sur les produits d'hydratation.

    """
    # Impact de la température
    if temp_jour < 10:  # Température froide
        temp_effect = 1.4  # +40% produits hydratants
    elif temp_jour > 30:  # Température chaude
        temp_effect = 1.2  # +20%
    else:
        temp_effect = 1.0

    # Impact de l'humidité
    if humidity_jour < 40:  # Faible humidité
        humidity_effect = 1.3  # Augmentation des ventes de produits hydratants
    elif humidity_jour > 80:  # Humidité très élevée
        humidity_effect = 0.8
    else:
        humidity_effect = 1.0

    # Impact du vent
    if wind_speed_jour > 20:  # Vent fort
        wind_effect = 1.3  # +30% produits de protection
    else:
        wind_effect = 1.0

    # Combinaison des effets
    return min(2.0, temp_effect * humidity_effect * wind_effect)

def calcul_impact_climatique(temp_jour, humidity_jour, wind_speed_jour):
    """Calcul de l'impact climatique spécifique aux produits d'hydratation"""
    return meteo_hydratation_impact(temp_jour, humidity_jour, wind_speed_jour)

def generer_ventes_hydratation(df, base_ventes=50):
    """Génère les ventes avec impact climatique de base"""
    # Calculer l'impact climatique
    impact_climatique = df.apply(
        lambda row: calcul_impact_climatique(
            row['temp_jour'],
            row['humidity_jour'],
            row['wind_speed_jour']
        ),
        axis=1
    )

    # Générer les ventes totales de produits d'hydratation
    df['ventes_produits_hydratation'] = impact_climatique * base_ventes
    # Limitation des ventes journalières
    max_ventes = 100
    df['ventes_produits_hydratation'] = df['ventes_produits_hydratation'].clip(upper=max_ventes)

    return df




def ajouter_bruit(df, niveau_bruit=0.1):
    """Ajoute un bruit réaliste aux données"""
    df_avec_bruit = df.copy()

    # Générer des facteurs multiplicatifs avec distribution normale
    bruit = np.random.normal(1.0, niveau_bruit, size=len(df))

    # Appliquer le bruit multiplicatif
    df_avec_bruit["ventes_produits_hydratation"] = df["ventes_produits_hydratation"] * bruit

    # S'assurer que toutes les valeurs sont positives
    df_avec_bruit["ventes_produits_hydratation"] = np.maximum(1, df_avec_bruit["ventes_produits_hydratation"])

    # Arrondir aux entiers
    df_avec_bruit["ventes_produits_hydratation"] = df_avec_bruit["ventes_produits_hydratation"].round().astype(int)

    return df_avec_bruit

def processus_de_generation_complet(df, base_ventes=30, niveau_bruit=0.1):
    """Processus complet de génération de données"""
    # 1. Génération des ventes de base avec impact climatique
    df = generer_ventes_hydratation(df, base_ventes)



    # 2. Ajout de bruit pour un modèle plus réaliste
    df = ajouter_bruit(df, niveau_bruit=niveau_bruit)

    # 3. Afficher les statistiques
    print_statistiques(df)

    return df

def print_statistiques(df):
    """Affiche les statistiques de vente pour vérification"""
    print("=== STATISTIQUES DES VENTES DE PRODUITS D'HYDRATATION ===")
    print(f"Ventes moyennes globales: {df['ventes_produits_hydratation'].mean():.2f}")


    # Par saison
    if 'saison' in df.columns:
        for saison in df['saison'].unique():
            print(f"Ventes moyennes pendant {saison}: {df.loc[df['saison'] == saison, 'ventes_produits_hydratation'].mean():.2f}")



    print("===============================")


df = processus_de_generation_complet(df, base_ventes=30, niveau_bruit=0.1)

=== STATISTIQUES DES VENTES DE PRODUITS D'HYDRATATION ===
Ventes moyennes globales: 31.37
Ventes moyennes pendant hiver: 30.99
Ventes moyennes pendant printemps/automne: 30.54
Ventes moyennes pendant été: 33.40


In [ ]:
print(df['ventes_produits_hydratation'].unique())

[29 36 31 30 32 33 25 28 27 34 35 26 37 24 23 21 38 41 43 40 45 39 42 44
 47 49 22 46 48 20 19 50 52 51 55 56 54 58 53 57 18 15 60 59 17 63]


# Modélisation des Ventes de Charbon : Approche Saisonnière et Régionale

Ce module génère des données synthétiques réalistes pour les ventes de charbon en fonction des saisons, des conditions météorologiques et des spécificités régionales.

---

## 1. Impact Saisonniers et Climatiques

### Coefficients Mensuels
| Période | Coefficient | Usage Principal |
|---------|-------------|-----------------|
| Hiver (déc-fév) | 1.7-1.8 | Chauffage |
| Été (juin-août) | 1.2-1.4 | Barbecues |
| Autres mois | 1.0 | - |

### Effet de la Température
| Saison | Conditions | Impact |
|--------|------------|--------|
| Hiver | <10°C | +5% par °C en dessous de 10°C |
| Été (zones côtières) | >25°C | +3% par °C au-dessus de 25°C |

---

## 2. Facteurs Comportementaux

### Weekends et Jours Fériés (Été)
- **Coefficient** : 1.5 (+50%)
- **Explication** : Augmentation des barbecues en période estivale

### Spécificités Régionales
| Type de Zone | Coefficient | Explication |
|--------------|-------------|-------------|
| Côtière | 1.5 | Forte activité touristique estivale |
| Hors côtier | 1.1 | - |
| Intérieure | 0.9 | Moindre usage estival |

---

## 3. Paramètres de Contrôle

- **Ventes de base** : 20 unités/jour
- **Maximum journalier** : 100 unités
- **Bruit aléatoire** : ±10% pour le réalisme
- **Plafond global** : +150% (coefficient max de 2.5)

---

In [ ]:

def meteo_charbon_impact(temp_jour, mois, weekend, jour_ferie,region):
    """
    Calcule l'impact météorologique et saisonnier sur les ventes de charbon.

    """
    # Coefficients saisonniers
    monthly_coeff = {
        1: 1.8, 2: 1.7, 12: 1.8,  # Hiver (chauffage)
        6: 1.2, 7: 1.3, 8: 1.4,   # Été (barbecues)
        **{m: 1.0 for m in [3, 4, 5, 9, 10, 11]}  # Autres mois
    }
    # Coefficients régionaux
    region_coeff = {
    'Côtière': 1.5,   # Zones touristiques (barbecues)
    'Hors côtier':1.1,
    'Intérieure':0.9
}


    # Impact de la température
    if mois in [12, 1, 2]:  # Hiver
        temp_effect = 1.0 + max(0, 0.05 * (10 - temp_jour)) if temp_jour < 10 else 1.0
    elif mois in [6, 7, 8] and region == 'Côtière':  # Été pour zones côtières
        temp_effect = 1.0 + max(0, 0.03 * (temp_jour - 25)) if temp_jour > 25 else 1.0
    else:
        temp_effect = 1.0

    # Impact des weekends et jours fériés
    weekend_holiday_effect = 1.5 if (mois in [6, 7, 8] and (weekend or jour_ferie)) else 1.0
    # Impact des coefficients régionaux
    region_effect = region_coeff.get(region, 1.0)


    # Combinaison des effets
    return min(2.5, monthly_coeff[mois] * temp_effect * weekend_holiday_effect * region_effect)

def calcul_impact_climatique_charbon(temp_jour, mois, weekend, jour_ferie , region):
    """Calcul de l'impact climatique spécifique aux ventes de charbon"""
    return meteo_charbon_impact(temp_jour, mois, weekend, jour_ferie , region)

def generer_ventes_charbon(df, base_ventes=50):
    """
    Génère les ventes de charbon avec impact climatique de base.
    """
    # Calculer l'impact climatique
    impact_climatique = df.apply(
        lambda row: calcul_impact_climatique_charbon(
            row['temp_jour'],
            row['mois'],
            row['Weekend'],
            row['jour_ferie'],
            row['profil_cotier']
        ),
        axis=1
    )

    # Générer les ventes totales de charbon
    df['ventes_charbon'] = impact_climatique * base_ventes

    # Limitation des ventes journalières
    max_ventes = 100
    df['ventes_charbon'] = df['ventes_charbon'].clip(upper=max_ventes)

    return df

def ajouter_bruit(df, niveau_bruit=0.1):
    """
    Ajoute un bruit réaliste aux données de ventes.

    """
    df_avec_bruit = df.copy()

    # Générer des facteurs multiplicatifs avec distribution normale
    bruit = np.random.normal(1.0, niveau_bruit, size=len(df))

    # Appliquer le bruit multiplicatif
    df_avec_bruit["ventes_charbon"] = df["ventes_charbon"] * bruit

    # S'assurer que toutes les valeurs sont positives
    df_avec_bruit["ventes_charbon"] = np.maximum(1, df_avec_bruit["ventes_charbon"])

    # Arrondir aux entiers
    df_avec_bruit["ventes_charbon"] = df_avec_bruit["ventes_charbon"].round().astype(int)

    return df_avec_bruit

def processus_de_generation_complet(df, base_ventes=20, niveau_bruit=0.1):
    """
    Processus complet de génération de données de ventes de charbon.
    """
    # 1. Génération des ventes de base avec impact climatique
    df = generer_ventes_charbon(df, base_ventes)

    # 2. Ajout de bruit pour un modèle plus réaliste
    df = ajouter_bruit(df, niveau_bruit=niveau_bruit)

    # 3. Afficher les statistiques
    print_statistiques(df)

    return df

def print_statistiques(df):
    """
    Affiche les statistiques de vente de charbon pour vérification.


    """
    print("=== STATISTIQUES DES VENTES DE CHARBON ===")
    print(f"Ventes moyennes globales: {df['ventes_charbon'].mean():.2f}")

    # Par saison
    if 'saison' in df.columns:
        for saison in df['saison'].unique():
            print(f"Ventes moyennes pendant {saison}: {df.loc[df['saison'] == saison, 'ventes_charbon'].mean():.2f}")

    print("===============================")


df = processus_de_generation_complet(df, base_ventes=20, niveau_bruit=0.1)

=== STATISTIQUES DES VENTES DE CHARBON ===
Ventes moyennes globales: 28.32
Ventes moyennes pendant hiver: 37.54
Ventes moyennes pendant printemps/automne: 21.51
Ventes moyennes pendant été: 32.81


In [ ]:
print(df['ventes_charbon'].unique())

[35 31 39 37 33 29 26 34 32 25 47 28 50 55 53 42 36 30 43 48 44 38 54 40
 41 27 46 51 49 52 57 23 24 16 19 17 15 21 18 22 20 12 14 13 45 60 59 56
 58 63 61 64 62 65 10]


In [ ]:
df=df.to_csv('data1.csv',index=False)

In [ ]:
df=pd.read_csv('data1.csv')
print(df.shape)

(43769, 59)


In [ ]:
import pandas as pd
df=pd.read_csv('data1.csv')
print(df.shape)

(43769, 54)


In [ ]:
print(df.columns)

Index(['date', 'city', 'profil_cotier', 'densite_urbaine', 'region_climatique',
       'temp_jour', 'humidity_jour', 'wind_speed_jour', 'precip_jour',
       'point de rosée/jour', 'temp_jour_lag1', 'humidity_jour_lag1',
       'wind_speed_jour_lag1', 'precip_jour_lag1', 'temp_jour_lag3',
       'humidity_jour_lag3', 'wind_speed_jour_lag3', 'precip_jour_lag3',
       'temp_jour_lag7', 'humidity_jour_lag7', 'wind_speed_jour_lag7',
       'precip_jour_lag7', 'mois', 'Jour', 'Weekend', 'jour_ferie', 'saison',
       'vacances_scolaires', 'ramadan', 'new_year', 'saison_toristique',
       'Indice de Chaleur', 'refroidissement éolien', 'Indice d’inconfort',
       'Indice de Vague de Chaleur', 'Indice de Vague de Froid ',
       'Indice de Pluie Intense', 'Indice de Tempête', 'Indice de sécheresse',
       'Indice climatique thermique universel (ICTU)', 'boissons_fraiches',
       'snacks_sucré', 'snacks_salé',
       'produits_laitiers_frais(yaourts, fromage blanc)',
       'produits_laiti

# **Conclusion**

Ce notebook a présenté une modélisation complète des ventes synthétiques de divers produits en fonction des conditions météorologiques et des facteurs saisonniers. Les principaux enseignements sont :

## Synthèse des Résultats

1. **Impact climatique significatif** :  
   Tous les produits analysés montrent une sensibilité marquée aux variations de température, d'humidité et des précipitations, avec des coefficients d'impact allant jusqu'à +50% pour certaines conditions extrêmes.

2. **Saisonnalité prononcée** :  
   Les modèles révèlent des variations saisonnières importantes, particulièrement pour :
   - Les produits de chauffage (hiver)
   - Les articles de plein air (été)
   - Les produits de soin (pics en périodes de transition)

3. **Spécificités régionales** :  
   Les zones côtières et urbaines présentent des comportements d'achat distincts, nécessitant des coefficients adaptés.

